# Shaft Grinder 

Useful how to's:

For rviz open:

roscd hrr_common/
❯ cd rviz
❯ rviz -d hrr_cobot_hw.rviz

plot forces:
rqt_plot /hrr_cobot/ft_sensor/wrench/force/x:y:z


### how to use tools:
* catkin build hrr_ee_tools package
* Start robot node(roslaunch)
* Start Arduino node (roslaunch hrr_ee_tools tool_controller.launch) 



In [1]:
# packages needed for this tutorial
from pathlib import Path
# math and robotics
import numpy as np
import quaternion
from tqdm.notebook import tqdm, trange
import spatialmath as sm

# plotting 
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt

# simulated robot content and helpers
import hrr_common as hrr_cm
import hrr_cobot_robot as hrr_rob
from hrr_cobot_robot.tool_control import ArduinoToolControlInterface
from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
from std_msgs.msg import Float32, Int8
from geometry_msgs.msg import Vector3

# ROS
import rospy

# set printing and plotting options    
np.set_printoptions(precision=5, suppress=True)
sns.set_theme('notebook')
%matplotlib notebook

In [2]:
%config Completer.use_jedi = False

In [3]:
%env ROS_MASTER_URI=http://hrrcobotLinux54.lsr.ei.tum.de:11311
%env ROS_IP=129.187.147.183

env: ROS_MASTER_URI=http://hrrcobotLinux54.lsr.ei.tum.de:11311
env: ROS_IP=129.187.147.183


In [5]:
rospy.init_node("hrr_cobot_shaftgrinder")

In [34]:
a = rospy.wait_for_message('DevType', Int8).data

In [49]:
#rospy.set_param("/device_type", rospy.wait_for_message('/device_type', Int8).data)
rospy.set_param("/device_center_topic", rospy.wait_for_message('device_center_topic', Vector3).y)


In [42]:
a = rospy.get_param("z1test")
print(2*a)

2


In [50]:
rospy.get_param("/device_center_topic")

0.0

In [5]:
rospy.set_param("/hrr_cobot/cmd_arduino", True)

In [21]:
rospy.set_param("~cmd_arduino", True)
rospy.set_param("~tool_frame", "shaftgrinder_tip")
rospy.set_param("~arduino_node_name", "/hrr_cobot/tool_controller")
rospy.set_param("~tool_name", "shaftgrinder")

In [6]:
#hrr_rob.utils.load_default_parameters("/hrr_cobot")

#hrr_rob.HrrCobotControl.init_ros(shaft_grinder_ctrl_srv_name = "cutting")
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

In [7]:
ToolType(ToolType.SHAFT_GRINDER)

type: 3

In [56]:
cobot.tool_controller

In [6]:
cobot._tf.T_A_B('hrr_cobot.shaftgrinder_tip', 'hrr_cobot.ee_link')

(array([ 0.    , -0.1555, -0.0945]),
 quaternion(0.270598050074698, -0.270598050074698, -0.653281482437526, -0.653281482437526))

In [13]:
Shaftgrinder_tf_PC_Tower_1 = np.array([[-0.26864, 0.55951, 0.78408],
                                       [0.67257, -0.47376, 0.5685],
                                       [0.68955, 0.68007, -0.24904]]
                                       )
Shaftgrinder_tf = np.array([[-0.707107, 0.707107, 0.],
                            [0, 0, 1],
                            [0.707107, 0.707107, 0]]
                            )

In [16]:
T = np.linalg.inv(Shaftgrinder_tf) @ Shaftgrinder_tf_PC_Tower_1
print(T)
print(Shaftgrinder_tf @ T)

[[ 0.67754  0.08525 -0.73053]
 [ 0.29763  0.87652  0.37833]
 [ 0.67257 -0.47376  0.5685 ]]
[[-0.26864  0.55951  0.78408]
 [ 0.67257 -0.47376  0.5685 ]
 [ 0.68955  0.68007 -0.24904]]


In [17]:
Shaftgrinder_tf_PC_Tower_2 = np.array([[-0.25151, 0.58966, 0.76749],
                                       [0.22861, -0.73435, 0.63911],
                                       [0.94047, 0.3362, 0.04989],
                                       ])
Shaftgrinder_tf = np.array([[-0.707107, 0.707107, 0.],
                            [0, 0, 1],
                            [0.707107, 0.707107, 0]]
                            )

In [18]:
T = np.linalg.inv(Shaftgrinder_tf) @ Shaftgrinder_tf_PC_Tower_2
print(T)
print(Shaftgrinder_tf @ T)

[[ 0.84286 -0.17922 -0.50742]
 [ 0.48717  0.65468  0.57797]
 [ 0.22861 -0.73435  0.63911]]
[[-0.25151  0.58966  0.76749]
 [ 0.22861 -0.73435  0.63911]
 [ 0.94047  0.3362   0.04989]]


In [ ]:
T_B_tool = sm.SE3(*toEE)
T_B_tool.A[:3, :3] = quaternion.as_rotation_matrix(q)
T_B_tool.A[:, 3] = [0.455, 0.04829, 0.65335, 1]

In [7]:
cobot.T_E_C_robot.inv()
T_E_C = sm.SE3(*self.E_p_EC)
T_E_C.A[:3, :3] = self.R_E_C

SE3:   1           0           0           0            
       0           1           0           0            
       0           0           1           0            
       0           0           0           1            
    

In [9]:
cobot.tool_controller
#cobot.tool_controller.rpm = 0
cobot.change_tool(None)
cobot.change_tool("shaftgrinder")
cobot.tool_controller.reset()

In [42]:
print(cobot.T_B_C_robot)
print(cobot.T_B_E_robot)
toEE = [0.036878, 0.1479008, -0.107603]

  -0.972695    0.231637    0.0144529   0.492877     
   0.231601    0.972802   -0.0042012   0.0943616    
  -0.015033   -0.000739179-0.999887    0.595248     
   0           0           0           1            

  -0.698019    0.67758     0.231637    0.455999     
   0.166737   -0.160796    0.972802   -0.0535392    
   0.696397    0.717656   -0.000739005 0.702851     
   0           0           0           1            



In [46]:
cobot.tool_controller.run_shaftgrinder(2.0, 'slow')

In [45]:
print(0.595248-(0.702851))

-0.107603


In [45]:
cobot.change_tool("shaftgrinder")
cobot.T_B_C_robot

SE3:   0.707802   -0.70641    -0.000809734 0.455481     
       0.00117185  2.78944e-05 0.999999   -0.20854      
      -0.70641    -0.707803    0.000847551 0.748848     
       0           0           0           1            
    

In [12]:
from hrr_cobot_robot.utils import tool2tool_type
tool2tool_type(cobot.tool)

3

In [13]:
from hrr_cobot_robot import ArduinoToolControlInterface
rospy.set_param("~arduino_node_name", "/hrr_cobot/tool_controller")
tool_controller = ArduinoToolControlInterface.from_ros()

In [57]:
tool_controller.tool = "shaftgrinder"

NameError: name 'tool_controller' is not defined

In [58]:
cobot.change_tool("shaftgrinder")

True

In [28]:
tool_controller.rpm = 4000

[WARN] [1641301515.730047]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1641301515.735382]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [51]:
tool_controller.vacuum = True

In [98]:
r = rospy.Rate(2)
for rpm in tqdm((30e3, 20e3, 10e3, 5e3, 1e3)):
    cobot.tool_controller.rpm = rpm
    r.sleep()

  0%|          | 0/5 [00:00<?, ?it/s]

[WARN] [1641293163.098384]: maximum value is 25000. Set value accordingly.
[WARN] [1641293165.019923]: minimum value is 3500 => set value to 0


In [19]:
cobot.tool_controller

/home/adaniels/_ros/hrr_ws/src/hrr_cobot/hrr_cobot_robot/src/hrr_cobot_robot/hrr_cobot_handle.py:901: UserWarning: current tool is set to , which has no dedicated tool-controller interface.
  warnings.warn(f"current tool is set to {self.tool}, "


In [8]:
cobot.change_tool("shaftgrinder")

True

In [15]:
cobot.change_tool("vacuum")

[WARN] [1644854791.966491]: vacuum gripper contains two tips. This contradicts default pipeline atm. Use 1


True

In [21]:
cobot.tool_controller.disable()

[WARN] [1644855182.318502]: minimum value is 3500 => set value to 0


In [23]:
cobot.tool_controller.enable_grinder()

In [14]:
cobot.tool_controller.run

[WARN] [1644930092.048337]: minimum value is 3500 => set value to 0


In [79]:
cobot.tool_controller.reset()

In [7]:
print(cobot.tool_controller._cur_tool)

3


In [8]:
toEE = np.array([0, -0.156, 0.094])
toEE[2] = -toEE[2]
print(toEE)
toEE, q = cobot._tf.T_A_B('hrr_cobot.shaftgrinder_tip', 'hrr_cobot.ee_link')
toEE[2] = -toEE[2]
print(toEE)

[ 0.    -0.156 -0.094]
[ 0.     -0.1555  0.0945]


In [16]:
cobot.tool_controller.vacuum = True

In [25]:
r = rospy.Rate(2)
T = int(50 * cobot.hz)
for t in trange(T):
    cobot.tool_controller.enable_grinder()
    r.sleep()

  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### set tool_controller interface

In [11]:
rospy.set_param("~arduino_node_name", "/hrr_cobot/tool_controller")
tool_controller = ArduinoToolControlInterface.from_ros()

#### Robot Status Check
check cobot status

In [6]:
print(cobot.F_max)
print(cobot)

80.0
Hrr-Cobot state:
FT-data:
current force:	[ 34.3927   -0.427   -30.00586] [N]
current torque:	[ 4.60449 20.10406 -0.59387] [Nm]
current wrench:	[ 34.40167  -0.39792 -30.04685   4.61655  20.10544  -0.58243] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[    9.21336     1.01956  -104.10632   -91.06585    85.87448 -1109.09016][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.45514 0.007   0.68243][m]
quat:		-0.18757 <[0.34148 0.61953 0.68146]>
tool-pos:	[0.45514 0.007   0.68243][m]
tool-rpy:	[-135.73557   13.47381  -90.05077][°]
robot-status:	ready
---


#### Param Serv Check
test if setting grinder service names has worked

In [6]:
print(cobot._set_control_mode)
print(cobot._pub_rpm)

#### run calibration routine

In [27]:
cobot.run_calibration_routine(file_name='/tmp/current_tool.npy', plot=True, figsize=(10, 10))

[ERROR] [1641302361.994959]: could not find file /tmp/current_tool.npy
[ERROR] [1641302367.946924]: cannot activate sensor-track velocity controller: The robot is currently in status: terminate => please restart


KeyboardInterrupt: 

#### Shaft Grinder Control
enable and disable grinder procedure

In [9]:
cobot.publish_rpm(7000)
# cobot.enable_grinder()

In [10]:
cobot.tool .enable_grinder()

In [11]:
cobot.reset_tool()

In [7]:
for t in trange(20):
    cobot.enable_grinder()
    rospy.sleep(0.2)

  0%|          | 0/20 [00:00<?, ?it/s]

In [12]:
cobot.disable_tool()

#### Vacuum Control

In [12]:
for t in trange(20):
    cobot.enable_vacuum()
    rospy.sleep(0.2)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
cobot.disable_tool()

In [ ]:
cobot.reset_tool()

#### Work with hrr_cobot_robot class
check different position forms

In [9]:
comau_status = hrr_rob.ComauRobotState.from_ros()
print(comau_status)
print(comau_status.pos)
print(cobot.FK(cobot.q))
print(sm.SE3(cobot.sns_pos))
print(cobot.sns_pos)

q:		[    1.02109    68.44272   -86.12142   -89.10544    90.36513 -2410.47627][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.60502 0.06922 0.17382][m]
quat:		0.27054 <[-0.27016  0.65383  0.65294]>
tool-pos:	[0.60502 0.06922 0.17382][m]
tool-rpy:	[135.04379   0.05643 -90.05444][°]
robot-status:	ready
[0.60502 0.06922 0.17382]
  -0.707646   -0.706566    0.000984772 0.605014     
   2.53283e-05 0.00136838  0.999999    0.069218     
  -0.706567    0.707646   -0.00095043  0.173818     
   0           0           0           1            

   1           0           0           0.605016     
   0           1           0           0.069218     
   0           0           1           0.173819     
   0           0           0           1            

[0.60502 0.06922 0.17382]


#### Spatial Math
working with SM

In [24]:
position1 = np.array([0.348, -0.035, 0.66047])
print(sm.SE3(position1).A[:3,:3])

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
T = sm.SE3.Rz(45, 'deg')
print(T)
T1 = (T@cobot.FK(cobot.q))
print(T1)
T2 = sm.SE3.Rx(45,'deg')
print(T2)
T3 = T2@T1
print(T3)

In [38]:
toolmatrix = sm.SE3(np.array([[1, 0, 0, 0],
                       [0, 0, -1, -0.002],
                       [0, 1, 0, -0.001],
                       [0,0,0,1]]))
print(toolmatrix)

   1           0           0           0            
   0           0          -1          -0.002        
   0           1           0          -0.001        
   0           0           0           1            



In [13]:
newpos1 = toolmatrix @ cobot.FK(cobot.q)
print(newpos1)

   1           0           0           0            
   0           0          -1           0            
   0           1           0           0            
   0           0           0           1            

  -0.998348   -0.0571615  -0.00583575  0.43552      
  -0.0542935   0.905233    0.421433    0.0212582    
  -0.018807    0.421053   -0.906841    0.545793     
   0           0           0           1            

  -0.998348   -0.0571615  -0.00583575  0.43552      
   0.018807   -0.421053    0.906841   -0.545793     
  -0.0542935   0.905233    0.421433    0.0212582    
   0           0           0           1            

  -0.998348   -0.0571615  -0.00583575  0.43552      
   0.018807   -0.421053    0.906841   -0.545793     
  -0.0542935   0.905233    0.421433    0.0212582    
   0           0           0           1            



#### Matrix Calculations without SM
working without SM

In [22]:
C = np.array([[-0.61, -0.61, 0.5, 0.43],
             [-0.32, -0.388, -0.864, 0.133],
             [ 0.717, -0.7, 0.04, 0.77],
             [0, 0, 0, 1]])
print(C)
B = np.array([[-1, -0.07, 0.02, 0.43],
              [0, 0.45, 0.89, -0.247],
              [-0.06, 0.88, -0.45, 0.481],
              [0, 0, 0, 1]])
print(B)
A = C @ np.linalg.inv(B)
print(A)

print(A@B)

[[-0.61  -0.61   0.5    0.43 ]
 [-0.32  -0.388 -0.864  0.133]
 [ 0.717 -0.7    0.04   0.77 ]
 [ 0.     0.     0.     1.   ]]
[[-1.    -0.07   0.02   0.43 ]
 [ 0.     0.45   0.89  -0.247]
 [-0.06   0.88  -0.45   0.481]
 [ 0.     0.     0.     1.   ]]
[[ 0.6539   0.17712 -0.73174  0.54454]
 [ 0.31598 -0.94403  0.06697 -0.26826]
 [-0.67505 -0.29338 -0.69913  1.32409]
 [ 0.       0.       0.       1.     ]]
[[-0.61  -0.61   0.5    0.43 ]
 [-0.32  -0.388 -0.864  0.133]
 [ 0.717 -0.7    0.04   0.77 ]
 [ 0.     0.     0.     1.   ]]


#### Optional: Activate Needed Controllers

In [ ]:
cobot.init_sns_vel()

In [ ]:
sns_trk_vel_cmd = hrr_rob.SnsTrkCmd.from_ros()
sns_trk_vel_cmd.list_controllers()

#### Optional: Check Robot Status again
if not working for example

In [185]:
print(cobot)
print(cobot.q_calib)
print(cobot.sns_pos)

Hrr-Cobot state:
command-state:	 cart_servo
FT-data:
current force:	[ -2.15881 -21.89795  18.24609] [N]
current torque:	[ 3.82538 -1.56372 -2.12097] [Nm]
current wrench:	[ -2.11093 -21.89827  18.23526   3.84715  -1.55118  -2.11185] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[   -4.63849    36.42405  -112.34628    -0.09231    31.17376 -1084.61197][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.51094 0.04152 0.27863][m]
quat:		0.00044 <[0.00046 1.      0.00044]>
tool-pos:	[0.51094 0.04152 0.27863][m]
tool-rpy:	[-179.94759    0.05031 -179.94943][°]
robot-status:	ready
--- ROBOT needs reset!
---
[ 0.      0.     -1.5708  0.      1.5708  0.    ]
[0.51094 0.04152 0.27863]


#### Move to Calib Pose
implements goTo(cobot.FK(cobot.q_calib))

In [33]:
cobot.move_to_calibration_pose()

[WARN] [1644939416.698723]: no calibration pose set for current tool shaft_grinder
[WARN] [1644939416.702734]: there is no check against calibration pose for the shaft grinder, use with care


True

#### GoTo command 
includes move_to command but has less input parameters 

In [34]:
cobot.goTo(cobot.FK(cobot.q_calib))

In [8]:
p, q = cobot._tf.T_A_B('hrr_cobot.ee_link', 'hrr_cobot.shaftgrinder_tip')
T_B_tool = sm.SE3(*p)
T_B_tool.A[:3, :3] = quaternion.as_rotation_matrix(q)
T_B_tool.A[:,3]= [0.455, 0.04829, 0.65335, 1]
new = T_B_tool #@ Rz
print(p)
print(new)

[0.06682 0.06682 0.1555 ]
  -0.707107    0           0.707107    0.455        
   0.707107    0           0.707107    0.04829      
   0           1           0           0.65335      
   0           0           0           1            



In [7]:
Rz = np.array([[0, -1, 0, 0],
                [1, 0, 0, 0],
                [0, 0, 1, 0],
                [0,0,0,1]])
p, q = cobot._tf.T_A_B('hrr_cobot.shaftgrinder_tip', 'hrr_cobot.ee_link')
T_B_tool = sm.SE3(*p)
T_B_tool.A[:3, :3] = quaternion.as_rotation_matrix(q)
T_B_tool.A[:,3]= [0.455, 0.04829, 0.65335, 1]
new = np.array(T_B_tool) @ Rz
print(p)
print(new)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 1)

#### MoveTo Command
1. set pose and velocities etc.
2. execute MoveTo

In [9]:
cobot.move_to_pose(Shaftgrinder_tf1,v_max=0.02,rot_max=0.05)

[ERROR] [1646152551.166354]: could not reach motion planner. Use linear interpolation instead


In [10]:
# commented for safety reasons -> outcomment for testing
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1646152564.995430]: reached goal pose at step 1239 / 5000


In [8]:
Shaftgrinder_tf = np.array([[-0.707107, 0.707107, 0., 0.455498],
                            [0, 0, 1, 0.0482934],
                            [0.707107, 0.707107, 0, 0.653348],
                            [0, 0, 0, 1]])
Shaftgrinder_tf1 = Shaftgrinder_tf @ Rz @ Rz

In [45]:
B=(A@Calib_Pose)
print(B)
print(Calib_Pose)
print(cobot.FK(cobot.q))
print(A)
print(Alt_TF)
deltacoord = np.array([[1, 0, 0, 0.6],
                       [0, 1, 0, 0.23],
                       [0, 0, 1, 0.16],
                       [0,0,0,1]])
print(deltacoord@B)
B[:,3]=[0.4, 0, 0.70, 1]
print(B)

deltacoord = np.array([[1, 0, 0, -0.117],
                       [0, 1, 0, 0.136],
                       [0, 0, 1, -0.072],
                       [0,0,0,1]])

[[-0.43992 -0.43973  0.78301 -0.06751]
 [ 0.5527   0.55465  0.62201 -0.38912]
 [-0.70781  0.7064   0.       0.02639]
 [ 0.       0.       0.       1.     ]]
[[-1.       0.       0.       0.43642]
 [ 0.       1.       0.       0.     ]
 [ 0.       0.      -1.       0.70451]
 [ 0.       0.       0.       1.     ]]
  -0.999999    0.00095575  0.000961403 0.435939     
   0.000956678 0.999999    0.000965388-0.000468488  
  -0.00096048  0.000966307-0.999999    0.704517     
   0           0           0           1            

[[ 0.43992 -0.43973 -0.78301  0.29215]
 [-0.5527   0.55465 -0.62201  0.2903 ]
 [ 0.70781  0.7064   0.      -0.28251]
 [ 0.       0.       0.       1.     ]]
[[-0.43992 -0.43973  0.78301  0.55597]
 [ 0.5527   0.55465  0.62201 -0.15123]
 [-0.70781  0.7064   0.       0.18   ]
 [ 0.       0.       0.       1.     ]]
[[-0.43992 -0.43973  0.78301  0.53249]
 [ 0.5527   0.55465  0.62201 -0.15912]
 [-0.70781  0.7064   0.       0.18639]
 [ 0.       0.       0.       1.     ]]
[[

In [79]:
B=(A[0:3,0:3]@Calib_Pose[0:3,0:3])
print(B)
print(Calib_Pose)
print(cobot.FK(cobot.q))
print(A)
print(Alt_TF)
deltacoord = np.array([[1, 0, 0, 0.6],
                       [0, 1, 0, 0.23],
                       [0, 0, 1, 0.16],
                       [0,0,0,1]])
v=Calib_Pose[:,3]+[-0.117, 0.136, -0.702,0]
vb=[0,0,0]
v = [1,2,3,4]
B = np.r_[B,[vb]]
B = np.c_[B,[v]]
#B = np.c_[B, [np.transpose(v)]]
#B = np.hstack([B,[vb]])
#B = np.vstack([B, [v]])

print(B)

deltacoord = np.array([[1, 0, 0, -0.117],
                       [0, 1, 0, 0.136],
                       [0, 0, 1, -0.072],
                       [0,0,0,1]])
print(np.transpose(v))

[[-0.43992 -0.43973  0.78301]
 [ 0.5527   0.55465  0.62201]
 [-0.70781  0.7064   0.     ]]
[[-1.       0.       0.       0.43642]
 [ 0.       1.       0.       0.     ]
 [ 0.       0.      -1.       0.70451]
 [ 0.       0.       0.       1.     ]]
  -0.725919    0.0709146   0.684114    0.400476     
   0.567556    0.623594    0.537597   -0.000469061  
  -0.388486    0.778525   -0.492927    0.700475     
   0           0           0           1            

[[ 0.43992 -0.43973 -0.78301  0.29215]
 [-0.5527   0.55465 -0.62201  0.2903 ]
 [ 0.70781  0.7064   0.      -0.28251]
 [ 0.       0.       0.       1.     ]]
[[-0.43992 -0.43973  0.78301  0.55597]
 [ 0.5527   0.55465  0.62201 -0.15123]
 [-0.70781  0.7064   0.       0.18   ]
 [ 0.       0.       0.       1.     ]]


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 4 and the array at index 1 has size 1

#### Set Transformation Matrix for Shaftgrinder

In [39]:
print(cobot.FK(cobot.q))

# Shaftgrinder_tf = np.array([[-0.707107, -0.707107, 0.238541, 0.455498],
#                        [0, 0, 1, 0.0482934],
#                        [-0.707107, 0.707107, 0, 0.653348],
#                        [0,0,0,1]])

Shaftgrinder_tf_1 = np.array([[-0.707107, 0.707107, 0.238541, 0.455498],
                            [0, 0, 1, 0.0482934],
                            [0.707107, 0.707107, 0, 0.653348],
                            [0, 0, 0, 1]])
print(Shaftgrinder_tf_1)

  -0.999839    0.000398775 0.0179324   0.58984      
   0.00024307  0.999962   -0.0086842  -0.0994496    
  -0.0179352  -0.00867845 -0.999801    0.57869      
   0           0           0           1            

[[-0.70711  0.70711  0.23854  0.4555 ]
 [ 0.       0.       1.       0.04829]
 [ 0.70711  0.70711  0.       0.65335]
 [ 0.       0.       0.       1.     ]]


In [10]:
Shaftgrinder_tf_PC_Tower_1 = np.array([[-0.26864, 0.55951, 0.78408, 0.50352],
                                       [0.67257, -0.47376, 0.5685, -0.04761],
                                       [0.68955, 0.68007, -0.24904, 0.26459],
                                       [0., 0., 0., 1.]])
Rz = sm.SE3.Rz(np.pi)
print(Rz)
Rz = np.array([[-1, 0, 0, 0],
                [0, -1, 0, 0],
                [0, 0, 1, 0],
                [0,0,0,1]])

PCt1 = Shaftgrinder_tf_PC_Tower_1 @ Rz
print(PCt1)

  -1           0           0           0            
   0          -1           0           0            
   0           0           1           0            
   0           0           0           1            

[[ 0.26864 -0.55951  0.78408  0.50352]
 [-0.67257  0.47376  0.5685  -0.04761]
 [-0.68955 -0.68007 -0.24904  0.26459]
 [ 0.       0.       0.       1.     ]]


In [8]:
Shaftgrinder_tf_1 = np.array([[-0.707107, 0.707107, 0, 0.455498],
                            [0, 0, 1, 0.0482934],
                            [0.707107, 0.707107, 0, 0.653348],
                            [0, 0, 0, 1]])

In [6]:
Shaftgrinder_tf = np.array([[-0.707107, -0.707107, 0, 0.455498],
                       [0, 0, 1, 0.0482934],
                       [-0.707107, 0.707107, 0, 0.653348],
                       [0,0,0,1]])

In [35]:
## A
print(cobot.FK(cobot.q))

  -0.999999    0.000838402-0.000763516 0.454691     
   0.000837611 0.999999    0.00103587 -0.23272      
   0.000764384 0.00103523 -0.999999    0.0542448    
   0           0           0           1            



In [46]:
## B
print(cobot.FK(cobot.q))

  -0.707437    0.706776   -0.000909616 0.656998     
   0.000389232 0.00167659  0.999999   -0.131722     
   0.706776    0.707436   -0.00146118  0.163471     
   0           0           0           1            



In [37]:
## C
print(cobot.FK(cobot.q))

  -0.999839    0.000393948 0.0179365   0.59925      
   0.000238131 0.999962   -0.00868844 -0.104008     
  -0.0179392  -0.00868277 -0.999801    0.0539813    
   0           0           0           1            



In [38]:
# Punkte wenn Fräser exakt 6cm raussteht:


In [53]:
## A z offset 5mm
print(cobot.FK(cobot.q))

  -0.715634    0.698224   -0.0187352   0.456093     
  -0.0415472  -0.0157771   0.999012   -0.390228     
   0.697238    0.715706    0.0402999   0.140382     
   0           0           0           1            



In [52]:
## B
print(cobot.FK(cobot.q))

  -0.715634    0.698224   -0.0187353   0.456093     
  -0.0415474  -0.0157772   0.999012   -0.390228     
   0.697239    0.715705    0.0403001   0.140382     
   0           0           0           1            



In [49]:
## C z offset 2mm
print(cobot.FK(cobot.q))

  -0.712013    0.702103   -0.00944944  0.597974     
  -0.0187505  -0.00555895  0.999809   -0.258931     
   0.701916    0.712054    0.0171228   0.156214     
   0           0           0           1            



In [54]:
## C z offset 2mm
print(cobot.FK(cobot.q))

  -0.717416    0.69605    -0.028791    0.455764     
  -0.0672226  -0.0280326   0.997344   -0.433634     
   0.693394    0.717446    0.0669013   0.278681     
   0           0           0           1            



In [32]:
Rz = sm.SE3.Rz(np.pi/2)
print(Rz)
Rz = np.array([[0, -1, 0, 0],
                [1, 0, 0, 0],
                [0, 0, 1, 0],
                [0,0,0,1]])
shaftgrindertf = Shaftgrinder_tf_1 @ Rz
print(shaftgrindertf)

   0          -1           0           0            
   1           0           0           0            
   0           0           1           0            
   0           0           0           1            

[[ 0.70711  0.70711  0.       0.4555 ]
 [ 0.       0.       1.       0.04829]
 [ 0.70711 -0.70711  0.       0.65335]
 [ 0.       0.       0.       1.     ]]


In [54]:
print(T_B_tool.A[:,3])
print(T_B_tool)

[ 0.     -0.1555 -0.0945  1.    ]
  -0.707107    0.707107    0           4.0401e-13   
   0           0           1          -0.1555       
   0.707107    0.707107    0          -0.0945       
   0.455       0.04829     0.65335     1            



In [9]:
p, q = cobot._tf.T_A_B('hrr_cobot.shaftgrinder_tip', 'hrr_cobot.ee_link')
Rz = sm.SE3.Rz(np.pi)
T_B_tool = sm.SE3(*p)
T_B_tool.A[:3, :3] = quaternion.as_rotation_matrix(q)
T_B_tool.A[:,3]= [0.455, 0.04829, 0.65335, 1]
new = T_B_tool @ Rz
print(new)

   0.707107   -0.707107    0           0.455        
   0           0           1           0.04829      
  -0.707107   -0.707107    0           0.65335      
   0           0           0           1            



In [38]:
Rz = sm.SE3.Rz(np.pi)
print(Rz)
Rz = np.array([[0, -1, 0, 0],
                [1, 0, 0, 0],
                [0, 0, 1, 0],
                [0,0,0,1]])
shaftgrindertf = Shaftgrinder_tf @ Rz
print(shaftgrindertf)

  -1           0           0           0            
   0          -1           0           0            
   0           0           1           0            
   0           0           0           1            

[[-0.70711  0.70711  0.       0.4555 ]
 [ 0.       0.       1.       0.04829]
 [ 0.70711  0.70711  0.       0.65335]
 [ 0.       0.       0.       1.     ]]


##### Alternative Transformation Matrix for Grinder (Robot shifted to the back a little bit)

In [7]:
Alt_TF = np.array([[-0.439919, -0.439733, 0.783011, 0.555965],
                       [0.552695, 0.554649, 0.622007, -0.151232],
                       [-0.707813, 0.706399, 0, 0.18],
                       [0,0,0,1]])

In [8]:
cobot.FK(cobot.q)

SE3:  -0.538323    0.563546    0.626597    0.455134     
       0.461285   -0.425211    0.778724   -0.00483921   
       0.705283    0.708246   -0.0310539   0.720699     
       0           0           0           1            
    

#### Calib_Pose

In [14]:
print(cobot.FK(cobot.q))
Calib_Pose = np.array([[-1, 0, 0, 0.436417 ],
                       [0, 1, 0, 0],
                       [0, 0, -1, 0.704513 ],
                       [0,0,0,1]])
print(Calib_Pose)

  -0.963395   -0.169999   -0.207293    0.29256      
  -0.206983    0.963081    0.17214     0.193581     
   0.170376    0.208745   -0.963015    0.511207     
   0           0           0           1            

[[-1.       0.       0.       0.43642]
 [ 0.       1.       0.       0.     ]
 [ 0.       0.      -1.       0.70451]
 [ 0.       0.       0.       1.     ]]


#### Calculating exact tf

In [53]:
#Marker on Table with shaft grinder:
print(cobot.FK(cobot.q))

  -0.43973    -0.440959    0.782428    0.439441     
   0.551584    0.554932    0.622741   -0.0157041    
  -0.708797    0.705412   -0.000794042 0.108471     
   0           0           0           1            



In [124]:
print(0.108471-0.0749837)

0.0334873


In [ ]:
# Used delta:
deltacoord = np.array([[1, 0, 0, -0.117],
                       [0, 1, 0, 0.136],
                       [0, 0, 1, -0.072],
                       [0,0,0,1]])

# and @Alt_TF

In [29]:
WS = np.array([[-0.439919, -0.439733, 0.783011, 0.4399441],
                [0.552695, 0.554649, 0.622007, -0.0157041],
                [-0.707813, 0.706399, 0, 0.108471],
                [0,0,0,1]])

In [30]:
WOS = np.array([[-1, 0, 0, 0.510936 ],
                [0, 1, 0, 0.0415216],
                [0, 0, -1, 0.0749837 ],
                [0,0,0,1]])

In [32]:
A = WS @ np.linalg.inv(WOS)
print(A)
print(A@WOS)

[[ 0.43992 -0.43973 -0.78301  0.29215]
 [-0.5527   0.55465 -0.62201  0.2903 ]
 [ 0.70781  0.7064   0.      -0.28251]
 [ 0.       0.       0.       1.     ]]
[[-0.43992 -0.43973  0.78301  0.43994]
 [ 0.5527   0.55465  0.62201 -0.0157 ]
 [-0.70781  0.7064   0.       0.10847]
 [ 0.       0.       0.       1.     ]]


In [120]:
#Marker on Table without shaft grinder:
print(cobot.FK(cobot.q))

  -0.999999    0.000917328 0.000911858 0.510936     
   0.000918159 0.999999    0.000910466 0.0415216    
  -0.000911022 0.000911302-0.999999    0.0749837    
   0           0           0           1            



In [ ]:
# Used delta
deltacoord = np.array([[1, 0, 0, 0.075],
                       [0, 1, 0, 0.042],
                       [0, 0, 1, -0.630],
                       [0,0,0,1]])
# and @Calib_Pose

#### new Alt_TF

In [160]:
Alt_TF = np.array([[-0.439919, -0.439733, 0.783011, -0.07149499999999992],
                       [0.552695, 0.554649, 0.622007, -0.0572257],
                       [-0.707813, 0.706399, 0, 0.0334873],
                       [0,0,0,1]])
print(Alt_TF)
Calib_Pose2 = np.array([[1, 0, 0, 0.436417 ],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0.704513 ],
                       [0,0,0,1]])

[[-0.43992 -0.43973  0.78301 -0.07149]
 [ 0.5527   0.55465  0.62201 -0.05723]
 [-0.70781  0.7064   0.       0.03349]
 [ 0.       0.       0.       1.     ]]


In [178]:
Alt_TF = np.array([[0.439919, -0.439733, -0.783011, -0.07149499999999992],
                       [-0.552695, 0.554649, -0.622007, -0.0572257],
                       [0.707813, 0.706399, 0, 0.0334873],
                       [0,0,0,1]])
Test2 = np.array([[-1, 0, 0, 0.43 ],
                  [0, 1, 0, 0],
                  [0, 0, -1, 0.707813 ],
                  [0,0,0,1]])
print(Alt_TF@Test2)
print(A)

[[-0.43992 -0.43973  0.78301 -0.43656]
 [ 0.5527   0.55465  0.62201 -0.73515]
 [-0.70781  0.7064   0.       0.33785]
 [ 0.       0.       0.       1.     ]]
[[ 0.43992 -0.43973 -0.78301  0.29215]
 [-0.5527   0.55465 -0.62201  0.2903 ]
 [ 0.70781  0.7064   0.      -0.28251]
 [ 0.       0.       0.       1.     ]]


In [180]:
print(Calib_Pose@A)
print(Test2)
print(Calib_Pose)

[[-0.43992  0.43973  0.78301  0.14427]
 [-0.5527   0.55465 -0.62201  0.2903 ]
 [-0.70781 -0.7064   0.       0.98702]
 [ 0.       0.       0.       1.     ]]
[[-1.       0.       0.       0.43   ]
 [ 0.       1.       0.       0.     ]
 [ 0.       0.      -1.       0.70781]
 [ 0.       0.       0.       1.     ]]
[[-1.       0.       0.       0.43642]
 [ 0.       1.       0.       0.     ]
 [ 0.       0.      -1.       0.70451]
 [ 0.       0.       0.       1.     ]]


#### Move Shaftgrinder from initial position to next one by giving the delta coordinates

In [117]:
deltacoord = np.array([[1, 0, 0, 0.075],
                       [0, 1, 0, 0.042],
                       [0, 0, 1, -0.630],
                       [0,0,0,1]])

neu = deltacoord @ Calib_Pose

In [8]:
cobot.move_to_pose(Shaftgrinder_tf_1,v_max=0.01,rot_max=0.01)
# sets the desired pos and velocities

In [9]:
aktuell = cobot.FK(cobot.q)
print(aktuell)

  -0.702982    0.711199   -0.0035331   0.586419     
  -0.00332811  0.00167812  0.999993   -0.0932889    
   0.7112      0.702989    0.00118726  0.620348     
   0           0           0           1            



In [10]:
print(Shaftgrinder_tf_1)

[[-0.70711  0.70711  0.23854  0.4555 ]
 [ 0.       0.       1.       0.04829]
 [ 0.70711  0.70711  0.       0.65335]
 [ 0.       0.       0.       1.     ]]


In [9]:
print(f"move robot along path of at least {np.linalg.norm(cobot.sns_pos- cobot.T_des.t):02.3f} m")

move robot along path of at least 0.382 m


In [10]:
# executes the loop to move the robot
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1641992196.488317]: reached goal pose at step 4989 / 5000


#### Routine: from Initial Position go down in Z, left/right in Y and up in Z

In [35]:
geherunter = np.array([[1, 0, 0, 0.15],
                       [0, 1, 0, -0.2],
                       [0, 0, 1, -0.49],
                       [0,0,0,1]])

neu = geherunter @ newtry
print(neu)
print(cobot.FK(cobot.q))

[[-0.70711 -0.70711  0.23854  0.6055 ]
 [ 0.       0.       1.      -0.15171]
 [-0.70711  0.70711  0.       0.16335]
 [ 0.       0.       0.       1.     ]]
  -0.707792   -0.70642     0.000970604 0.605024     
   1.53631e-05 0.00135858  0.999999   -0.15123      
  -0.706421    0.707792   -0.000950741 0.202873     
   0           0           0           1            



In [36]:
geherüber = np.array([[1, 0, 0, 0.15],
                       [0, 1, 0, -0.02],
                       [0, 0, 1, -0.505],
                       [0,0,0,1]])

neurüber = geherüber @ newtry
print(neu)
print(cobot.FK(cobot.q))

[[-0.70711 -0.70711  0.23854  0.6055 ]
 [ 0.       0.       1.       0.00829]
 [-0.70711  0.70711  0.       0.14835]
 [ 0.       0.       0.       1.     ]]
  -0.707558   -0.706654    0.000982866 0.605016     
   1.95279e-05 0.00137132  0.999999    0.00877124   
  -0.706655    0.707558   -0.000956488 0.172865     
   0           0           0           1            



In [36]:
geherauf = np.array([[1, 0, 0, 0.15],
                       [0, 1, 0, -0.2],
                       [0, 0, 1, -0.47],
                       [0,0,0,1]])
neunachoben = geherauf @ newtry
print(neunachoben)
print(cobot.FK(cobot.q))

[[-0.70711 -0.70711  0.23854  0.6055 ]
 [ 0.       0.       1.      -0.15171]
 [-0.70711  0.70711  0.       0.18335]
 [ 0.       0.       0.       1.     ]]
  -0.707792   -0.70642     0.000970604 0.605024     
   1.53631e-05 0.00135858  0.999999   -0.15123      
  -0.706421    0.707792   -0.000950741 0.202873     
   0           0           0           1            



In [34]:
# commented for safety reasons -> outcomment for testing
cobot.move_to_pose(neu,v_max=0.002,rot_max=0.002)
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

# if cobot.state == None:
#     cobot.move_to_pose(neurüber, vmax=0.001,rot_max=0.001)
#     T = int(50 * cobot.hz)
#     for t in trange(T):
#         if cobot.state is None:
#             rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
#             break
#         elif cobot.state == "error":
#             rospy.logerr(f"robot in ERROR state")
#         cobot.update()
        
if cobot.state == None:
    cobot.move_to_pose(neunachoben, vmax=0.002,rot_max=0.002)
    T = int(50 * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        cobot.update()


  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1638982742.154302]: reached goal pose at step 408 / 5000


  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1638982744.485331]: reached goal pose at step 227 / 5000


#### Manually Activate Velocity Controller

In [15]:
sns_trk_vel_cmd = hrr_rob.SnsTrkCmd.from_ros()

In [16]:
sns_trk_vel_cmd.activate()

#### Move with certain velocity in Shaftgrinder_tip frame

In [75]:
r = rospy.Rate(50)
v_test = np.zeros(6)
for t in trange(500):
    v_test[5] = -6e-3 #* np.sin(t/ 100.0 * 2 * np.pi)
    sns_trk_vel_cmd.update_cmd(v_test, "shaftgrinder_tip")
    r.sleep() 
sns_trk_vel_cmd.stop()

  0%|          | 0/500 [00:00<?, ?it/s]

In [18]:
r = rospy.Rate(50)
v_test = np.zeros(6)
for t in trange(200):
    v_test[2] = 6e-3 #* np.sin(t/ 100.0 * 2 * np.pi)
    sns_trk_vel_cmd.update_cmd(v_test, "shaftgrinder_tip")
    r.sleep() 
sns_trk_vel_cmd.stop()

  0%|          | 0/200 [00:00<?, ?it/s]

#### Move to Single Point (for Safety reasons for example)

In [105]:
print(cobot.FK(cobot.q))
gotoPointnew = np.array([[1, 0, 0, 0.02],
                       [0, 1, 0, 0.185],
                       [0, 0, 1, -0.038],
                       [0,0,0,1]])
Alt_TF = np.array([[-0.439919, -0.439733, 0.783011, 0.555965],
                       [0.552695, 0.554649, 0.622007, -0.151232],
                       [-0.707813, 0.706399, 0, 0.18],
                       [0,0,0,1]])
point = gotoPointnew @ Alt_TF
print(point)

  -0.440069   -0.438544    0.783593    0.576444     
   0.553794    0.554375    0.621273    0.0332865    
  -0.70686     0.707353   -0.00109972  0.145475     
   0           0           0           1            

[[-0.43992 -0.43973  0.78301  0.57597]
 [ 0.5527   0.55465  0.62201  0.03377]
 [-0.70781  0.7064   0.       0.142  ]
 [ 0.       0.       0.       1.     ]]


In [102]:
gotoPoint = np.array([[1, 0, 0, 0.1],
                       [0, 1, 0, -0.2],
                       [0, 0, 1, -0.485],
                       [0,0,0,1]])

singlepoint = gotoPoint @ Shaftgrinder_tf
print(singlepoint)
print(cobot.FK(cobot.q))

NameError: name 'Shaftgrinder_tf' is not defined

In [8]:
cobot.move_to_pose(Alt_TF,v_max=0.005,rot_max=0.005)
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

[ERROR] [1641308108.660848]: cannot activate sensor-track velocity controller: The robot is currently in status: 69 => please restart


  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### Another 3 Point Routine (PointA, PointB, PointC)

In [17]:
print(cobot.FK(cobot.q))
gotoPointnew = np.array([[1, 0, 0, 0.02],
                       [0, 1, 0, 0.185],
                       [0, 0, 1, 0],
                       [0,0,0,1]])
ausdemWeg = np.array([[-0.439919, -0.439733, 0.783011, 0.555965],
                       [0.552695, 0.554649, 0.622007, -0.151232],
                       [-0.707813, 0.706399, 0, 0.18],
                       [0,0,0,1]])
pointA = gotoPointnew @ ausdemWeg


  -0.44006    -0.438557    0.783591    0.576445     
   0.55378     0.554387    0.621275    0.0332867    
  -0.706877    0.707335   -0.00109916  0.179529     
   0           0           0           1            



In [18]:
print(cobot.FK(cobot.q))
gotoPointnew = np.array([[1, 0, 0, 0.02],
                       [0, 1, 0, 0.185],
                       [0, 0, 1, -0.046],
                       [0,0,0,1]])
ausdemWeg = np.array([[-0.439919, -0.439733, 0.783011, 0.555965],
                       [0.552695, 0.554649, 0.622007, -0.151232],
                       [-0.707813, 0.706399, 0, 0.18],
                       [0,0,0,1]])
pointB = gotoPointnew @ ausdemWeg


  -0.44006    -0.438557    0.783591    0.576445     
   0.55378     0.554387    0.621275    0.0332867    
  -0.706877    0.707335   -0.00109916  0.179529     
   0           0           0           1            



In [19]:
print(cobot.FK(cobot.q))
gotoPointnew = np.array([[1, 0, 0, 0.02],
                       [0, 1, 0, 0.185],
                       [0, 0, 1, 0],
                       [0,0,0,1]])
ausdemWeg = np.array([[-0.439919, -0.439733, 0.783011, 0.555965],
                       [0.552695, 0.554649, 0.622007, -0.151232],
                       [-0.707813, 0.706399, 0, 0.18],
                       [0,0,0,1]])
pointC = gotoPointnew @ ausdemWeg


  -0.44006    -0.438557    0.783591    0.576445     
   0.55378     0.554387    0.621275    0.0332867    
  -0.706877    0.707335   -0.00109916  0.179529     
   0           0           0           1            



In [24]:
cobot.reset_tool()
cobot.publish_rpm(17000)
cobot.enable_grinder()

In [21]:
# commented for safety reasons -> outcomment for testing
cobot.move_to_pose(pointA,v_max=0.001,rot_max=0.001)
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()


if cobot.state == None:
    cobot.move_to_pose(pointB, vmax=0.0005,rot_max=0.0005)
    T = int(50 * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        cobot.enable_grinder()
        cobot.update()
        
if cobot.state == None:
    cobot.move_to_pose(pointC, vmax=0.001,rot_max=0.001)
    T = int(50 * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        cobot.enable_grinder()
        cobot.update()


  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1640270170.399956]: reached goal pose at step 2 / 5000


  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### ToDo: Include Automatic enable and disable grinder

In [11]:
cobot.enable_grinder()
cobot.publish_rpm(7000)

In [12]:
cobot.disable_grinder()

In [38]:
print(aktuell)
print(Shaftgrinder_tf)

  -0.603363    0.767597    0.216213    0.455043     
   0.160222   -0.148912    0.975784    0.0159017    
   0.781205    0.623394   -0.0331378   0.652853     
   0           0           0           1            

[[-0.70711 -0.70711  0.23854  0.4555 ]
 [ 0.       0.       1.       0.04829]
 [-0.70711  0.70711  0.       0.65335]
 [ 0.       0.       0.       1.     ]]


In [36]:
aktuell2 = cobot.FK(cobot.q)
print(aktuell2)

  -0.603204    0.767719    0.216223    0.455009     
   0.159921   -0.149174    0.975793   -0.0540378    
   0.78139     0.623181   -0.0327923   0.65289      
   0           0           0           1            



#### Calc Goal pose from Normal Vector and Point

In [10]:
normalA = np.array([0,0,-1])
pointA = np.array([0.4555,0.04829,0.65335])
normalB = np.array([0,0,-1])
pointB = np.array([0.4555,0.04029,0.65335])
toEE = np.array([0,0,0])
print(Shaftgrinder_tf[:3,:3])

NameError: name 'Shaftgrinder_tf' is not defined

In [39]:
normalA = np.array([0,0,-1])
pointA = np.array([0.4555,0.0159017,0.65335])
normalB = np.array([0,0,-1])
pointB = np.array([0.4555,-0.0540378,0.65335])
toEE = np.array([0,0,0])
print(Shaftgrinder_tf[:3,:3])

[[-0.70711 -0.70711  0.23854]
 [ 0.       0.       1.     ]
 [-0.70711  0.70711  0.     ]]


In [20]:
hrr_cm.calc_goal_pose(normal=normal,p_location=point)

SE3:   1           0           0           0.4555       
       0           1           0           0.04829      
       0           0           1           0.65335      
       0           0           0           1            
    

In [18]:
test = hrr_cm.calc_EE_goal_pose(normal=normal, p_location=point,C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3,:3])

In [15]:
test2 = hrr_cm.calc_EE_pre_pose_neg(normal=normal, p_location=point,C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3,:3],safety_distance=-0.05)

In [19]:
cobot.move_to_pose(test,v_max=0.005,rot_max=0.005)
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1639649252.695986]: reached goal pose at step 448 / 5000


In [57]:
def MoveLoop(pose,v_max=0.005,rot_max=0.005,timeout=50):
    
    cobot.move_to_pose(pose,v_max,rot_max)
    T = int(timeout * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        cobot.update()

#### Complete Action

* Determine Cutting Type
* Calculate all Poses
* go to PrePose A (GoTo) - Salman's Planner
* Turn Grinder ON
* Move Slowly to Cutting Pose A (MoveTo - Feedback - Loop?)
* Move Slowly to Cutting Pose B (MoveTo - Feedback - Loop?)
* Move Slowly to Prepose B (MoveTo)
* Turn Grinder OFF

In [18]:
Shaftgrinder_tf = np.array([[-0.707107, -0.707107, 0.238541, 0.455498],
                       [0, 0, 1, 0.0482934],
                       [-0.707107, 0.707107, 0, 0.653348],
                       [0,0,0,1]])

In [8]:
def MoveLoop(pose,v_max=0.005,rot_max=0.005,timeout=50):
    
    cobot.move_to_pose(pose,v_max,rot_max)
    T = int(timeout * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        cobot.update()

In [60]:
normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.4555,0.04829,0.65335], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.4555,0.03029,0.65335], "pointB")
#toEE = np.array([0,0,0])
#print(Shaftgrinder_tf[:3,:3])

In [22]:
toEE = np.array([0,0,0])
print(Shaftgrinder_tf[:3,:3])

[[-0.70711 -0.70711  0.23854]
 [ 0.       0.       1.     ]
 [-0.70711  0.70711  0.     ]]


In [5]:
material= MaterialType()
material = material.PLASTIC
materialmsg = MaterialType()
rpm = 0

# Cutting Type
if material == materialmsg.PLASTIC:
    rpm = 17000
elif material == materialmsg.CABLE:
    rpm = 17000
elif material == materialmsg.METAL:
    rpm = 17000
else:
    print("Error: no valid material given")
    
    
print(rpm)

17000


* surface_normal_start := normalA
* surface_normal_end := normalB
* start_location := pointA
* end_location := pointB

In [23]:
# Calculate Poses
PrePoseA = hrr_cm.calc_EE_pre_pose_neg(normal=normalA,
                                       p_location=pointA,
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3,:3],
                                       safety_distance=-0.05)

PrePoseB = hrr_cm.calc_EE_pre_pose_neg(normal=normalB,
                                       p_location=pointB,
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3,:3],
                                       safety_distance=-0.05)

CuttingPoseA = hrr_cm.calc_EE_pre_pose(normal=normalA,
                                       p_location=pointA,
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3,:3],
                                       safety_distance=0.005)

CuttingPoseB = hrr_cm.calc_EE_pre_pose(normal=normalB,
                                       p_location=pointB,
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3,:3],
                                       safety_distance=0.005)

TypeError: unsupported operand type(s) for *: 'Vector3Stamped' and 'float'

In [19]:
rpm = 7000

In [8]:
# Action
MoveLoop(pose=PrePoseA,v_max=0.05,rot_max=0.1,timeout=50)
cobot.enable_grinder()
cobot.publish_rpm(rpm)
MoveLoop(pose=CuttingPoseA,v_max=0.05,rot_max=0.1,timeout=50)
MoveLoop(pose=CuttingPoseB,v_max=0.05,rot_max=0.1,timeout=50)
MoveLoop(pose=PrePoseB,v_max=0.05,rot_max=0.1,timeout=50)
cobot.disable_grinder()

NameError: name 'MoveLoop' is not defined

## Video Intro

In [52]:
cobot.change_tool("shaftgrinder")

True

In [ ]:
cobot.goTo(cobot.FK(cobot.q_calib))

In [ ]:
cobot.F_max = 120 ##adjust if needed but talk to me/Volker if you need higher values 

In [ ]:
def MoveLoop(pose,v_max=0.005,rot_max=0.005,timeout=50,rpm=0.005):
    """
    Sends robot to given position.
    """
    cobot.move_to_pose(pose,v_max,rot_max,rot_precision_threshold=0.001, pos_precision_threshold=0.001)
    T = int(timeout * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        if rpm != 0:
            cobot.tool_controller.rpm = rpm
            cobot.tool_controller.enable_grinder()

        cobot.update()

In [ ]:
## sets a safe transformation for robot with shaftgrinder attached.

Shaftgrinder_tf = np.array([[-0.707107, 0.707107, 0.238541, 0.455498],
                            [0, 0, 1, 0.0482934],
                            [0.707107, 0.707107, 0, 0.653348],
                            [0, 0, 0, 1]])

## sets translational vector
toEE = np.array([0, -0.156, 0.094])

In [ ]:
# use data from vision/task planner or set coordinates manually

xstart = 0.592829 #determine start value next to cable (in start cutting pose)
ystart = -0.229713+0.10
zstart =  0.367

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstart,ystart,zstart], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstart,ystart+0.031,zstart], "pointB")

In [ ]:
PrePoseA = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalA.vector),
                                        p_location=hrr_cm.vec32np(pointA.vector),
                                        C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                        safety_distance=-0.05)

PrePoseB = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalB.vector),
                                       p_location=hrr_cm.vec32np(pointB.vector),
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                       safety_distance=-0.05)

CuttingPoseA = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalA.vector),
                                        p_location=hrr_cm.vec32np(pointA.vector),
                                        C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                        safety_distance=-0)

CuttingPoseB = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalB.vector),
                                       p_location=hrr_cm.vec32np(pointB.vector),
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                       safety_distance=-0)

In [ ]:
MoveLoop(pose=PrePoseA,v_max=0.3,rot_max=0.6,timeout=5000,rpm=0)
MoveLoop(pose=CuttingPoseA,v_max=0.3,rot_max=0.6,timeout=5000,rpm=20000)
MoveLoop(pose=CuttingPoseB,v_max=0.3,rot_max=0.6,timeout=5000,rpm=20000)
MoveLoop(pose=PrePoseB,v_max=0.3,rot_max=0.6,timeout=5000,rpm=20000)
cobot.tool_controller.rpm = 0
cobot.tool_controller.disable()

In [ ]:
cobot.goTo(cobot.FK(cobot.q_calib))

## Testing Purposes

In [9]:
cutting=hrr_rob.Cutting()
#cutting.init_ros('hrr')
cutting._from_ros()
print(cutting.cobot)

NameError: name 'out' is not defined

In [ ]:
normalA = np.array([0,0,-1])
pointA = np.array([0.4555,0.0159017,0.65335])
normalB = np.array([0,0,-1])
pointB = np.array([0.4555,-0.0540378,0.65335])
toEE = np.array([0,0,0])
print(Shaftgrinder_tf[:3,:3])

In [14]:
print(hrr_cm.vector3Stamped2np(normalA))
print(normalA.vector)
print(hrr_cm.vec32np(normalA.vector))

None
x: 0
y: 0
z: -1
[ 0  0 -1]


In [ ]:
hrr_cm.homog2quat

In [ ]:
dispose_location_quat = cu.homog2quat(dispose_location)
dispose_location_pos = cu.homog2pos(dispose_location)

In [8]:
material= MaterialType()
material = material.PLASTIC

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.4555,0.04829,0.65335], "pointA")
normalB = hrr_cm.ros_utils.np2vector3Stamped(np.r_[0,0,-1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.4555,0.03029,0.65335], "pointB")

AttributeError: module 'hrr_common.ros_utils' has no attribute 'np2Vector3Stamped'

In [6]:
rospy.set_param("~cmd_arduino", True)
rospy.set_param("~arduino_node_name", "/hrr_cobot/tool_controller")

In [54]:
cobot.FK(cobot.q)

Mujoco not found on this client. Do not use this file!
Mujoco not found on this client. Do not use this file!
current implementation does not cope with trees with edge degree over 1. Select first element from: ee_linkconnect_flange_attach_joint, ee_to_tcp  


SE3:   0.706414   -0.707798    0.000966801 0.455481     
       3.08604e-06 0.00136901  0.999999   -0.20854      
      -0.707799   -0.706413    0.00096927  0.785977     
       0           0           0           1            
    

In [11]:
!rostopic list | grep cutting

/cutting_server/cutting/cancel
/cutting_server/cutting/feedback
/cutting_server/cutting/goal
/cutting_server/cutting/result
/cutting_server/cutting/status


In [53]:
T_E_C = sm.SE3(*cobot.E_p_EC)
print(T_E_C)

   1           0           0           0            
   0           1           0           0            
   0           0           1           0            
   0           0           0           1            



In [76]:
Rz1 = np.array([[-1, 0, 0],
                [0, -1, 0],
                [0, 0, 1]])
Rz2 = np.array([[-1, 0, 0],
                [0, -1, 0],
                [0, 0, 1]])

In [83]:
transform1 = np.array([[0.67754, 0.08525, -0.73053],
                                       [0.29763, 0.87652, 0.37833],
                                       [0.67257, -0.47376, 0.5685]])
T_B_C = sm.SE3(*cobot.E_p_EC)
T_E_C.A[:3, :3] = cobot.R_E_C @ transform1
print(T_E_C)
print(cobot.T_E_C_robot)
print(np.array(cobot.T_B_C_robot.A[:3, :3]) @ transform1)
Shaftgrinder_tf_PC_Tower_1 = np.array([[-0.26864, 0.55951, 0.78408],
                                       [0.67257, -0.47376, 0.5685],
                                       [0.68955, 0.68007, -0.24904]])
print(Shaftgrinder_tf_PC_Tower_1 @ Rz1)

   0.67754     0.08525    -0.73053     0            
   0.29763     0.87652     0.37833     0            
   0.67257    -0.47376     0.5685      0            
   0           0           0           1            

   1           0           0           0            
   0           1           0           0            
   0           0           1           0            
   0           0           0           1            

[[-0.26732  0.55971  0.78439]
 [ 0.67216 -0.47496  0.56798]
 [ 0.69046  0.67907 -0.24925]]
[[ 0.26864 -0.55951  0.78408]
 [-0.67257  0.47376  0.5685 ]
 [-0.68955 -0.68007 -0.24904]]


In [90]:
PC1= np.array([[ 0.26864, -0.55951,  0.78408],
                [-0.67257,  0.47376,  0.5685 ],
                 [-0.68955, -0.68007, -0.24904]])
T = PC1 @ np.linalg.inv(np.array(cobot.T_B_C_robot.A[:3, :3]))
print(T)
Y = cobot.T_B_C_robot.A
H = T@cobot.T_B_C_robot.A[:3, :3]
Y[:3,:3] = H
print(Y)
print(cobot.T_B_C_robot)

[[ 0.58655  0.78331  0.20586]
 [-0.80989  0.56915  0.14191]
 [-0.00601 -0.24997  0.96823]]
[[ 0.26864 -0.55951  0.78408  0.45548]
 [-0.67257  0.47376  0.5685  -0.20854]
 [-0.68955 -0.68007 -0.24904  0.79685]
 [ 0.       0.       0.       1.     ]]
   0.706421   -0.707791    0.000971787 0.455482     
  -4.22912e-06 0.00136876  0.999999   -0.208539     
  -0.707792   -0.706421    0.000963931 0.796854     
   0           0           0           1            



In [92]:
PC2 = np.array([[-0.25151, 0.58966, 0.76749],
               [0.22861, -0.73435, 0.63911],
               [0.94047, 0.3362, 0.04989]])
x = np.array([[-1, 0, 0],
             [0, 1, 0],
              [0, 0 , -1]])
T = PC2 @ np.linalg.inv(np.array(cobot.T_B_C_robot.A[:3, :3]))
print(T)
Y = cobot.T_B_C_robot.A
H = T@cobot.T_B_C_robot.A[:3, :3]
Y[:3,:3] = H
print(Y)
print(cobot.T_B_C_robot)

[[-0.59428  0.7683  -0.23779]
 [ 0.68188  0.6381   0.35757]
 [ 0.42646  0.05035 -0.90311]]
[[-0.25151  0.58966  0.76749  0.45548]
 [ 0.22861 -0.73435  0.63911 -0.20854]
 [ 0.94047  0.3362   0.04989  0.79685]
 [ 0.       0.       0.       1.     ]]
   0.706421   -0.707791    0.000971787 0.455482     
  -4.22912e-06 0.00136876  0.999999   -0.208539     
  -0.707792   -0.706421    0.000963931 0.796854     
   0           0           0           1            



In [13]:
PC1= np.array([[ 0.26864, -0.55951,  0.78408],
                [-0.67257,  0.47376,  0.5685 ],
                 [-0.68955, -0.68007, -0.24904]])
x = np.array([[-1, 0, 0],
             [0, 1, 0],
              [0, 0 , -1]])
T = PC1 @ np.linalg.inv(x)
print(T)
Y = cobot.T_B_C_robot.A
H = T@cobot.T_B_C_robot.A[:3, :3]
Y[:3,:3] = H
print(Y)
print(cobot.T_B_C_robot)

[[-0.26864 -0.55951 -0.78408]
 [ 0.67257  0.47376 -0.5685 ]
 [ 0.68955 -0.68007  0.24904]]
[[-0.29865 -0.33326  0.89429  0.4565 ]
 [-0.01425  0.9385   0.34497  0.01689]
 [-0.95425  0.09029 -0.28504  0.70236]
 [ 0.       0.       0.       1.     ]]
  -0.587361    0.782993   -0.204765    0.456496     
   0.809312    0.569686   -0.14308     0.0168933    
   0.00462086 -0.249758   -0.968297    0.702358     
   0           0           0           1            



### Action:

#### How To:

* start all roslaunches (3)
* cutting_skill.py debug start (or normal start)
* Go Through following notebook stuff (set appropriate points)
* Click through debug modus in PyCharm

TODO: video, new transformation, to EE?, look at config yaml stuff and update todos

In [9]:
material= MaterialType()
material = material.PLASTIC

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.4555,0.0159017,0.65335], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.4555,-0.0540378,0.65335], "pointB")

In [20]:
shaftgrinder_client=actionlib.SimpleActionClient('/cutting_server/cutting', CuttingAction)
shaftgrinder_client.wait_for_server()
print('found')

found


In [21]:
goal=CuttingGoal()
goal.surface_normal_start=normalA
goal.surface_normal_end=normalB
goal.start_location=pointA
goal.end_location=pointB
goal.material.material_type=material
goal.timeout=200
goal.thickness = 0.005

print(goal)
shaftgrinder_client.send_goal(goal)
shaftgrinder_client.wait_for_result()
shaftgrinder_client.get_result()

surface_normal_start: 
  header: 
    seq: 0
    stamp: 
      secs: 1646154196
      nsecs: 646218299
    frame_id: "normalA"
  vector: 
    x: 0
    y: 0
    z: 1
surface_normal_end: 
  header: 
    seq: 0
    stamp: 
      secs: 1646154196
      nsecs: 646971225
    frame_id: "normalB"
  vector: 
    x: 0
    y: 0
    z: 1
start_location: 
  header: 
    seq: 0
    stamp: 
      secs: 1646154196
      nsecs: 646630048
    frame_id: "pointA"
  vector: 
    x: 0.4555
    y: 0.0159017
    z: 0.65335
end_location: 
  header: 
    seq: 0
    stamp: 
      secs: 1646154196
      nsecs: 647337436
    frame_id: "pointB"
  vector: 
    x: 0.4555
    y: -0.0540378
    z: 0.65335
material: 
  material_type: 1
thickness: 0.005
timeout: 200


KeyboardInterrupt: 

In [37]:
shaftgrinder_client.cancel_all_goals()

In [59]:
cobot.tool_controller.disable()

[WARN] [1644863120.376611]: minimum value is 3500 => set value to 0


In [8]:
msg = CuttingFeedback()
msg.state = cobot.cobot_state_msg("cutting", status=CobotState.BUSY)
msg

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-8-eb16480112c0>, line 2)

In [39]:
deviceType = 1

In [40]:
 def updatetf(y):
        """
        Updates the coordinate transformation matrix for the robot/shaftgrinder according to the device and cutting position.
        Args:
            y [float] : y coordinate for Cutting Pose A

        Returns: None

        """
        toEE, q = cobot._tf.T_A_B('hrr_cobot.shaftgrinder_tip', 'hrr_cobot.ee_link')
        Rz = sm.SE3.Rz(np.pi)
        T_B_tool = sm.SE3(*toEE)
        T_B_tool.A[:3, :3] = quaternion.as_rotation_matrix(q)
        T_B_tool.A[:, 3] = [0.455, 0.04829, 0.65335, 1]
        Shaftgrinder_tf = T_B_tool @ Rz
        toEE = np.array(toEE)

        if deviceType == 3:  # 3 for PC_tower
            if (y - self._yofcenter) < 0.06:
                tf = Shaftgrinder_tf_PC_Tower_1 @ Rz
            else:
                tf = Shaftgrinder_tf_PC_Tower_2 @ Rz
        else:
            tf = Shaftgrinder_tf
        return tf, toEE

In [47]:
start_location = goal.start_location
surface_normal_start = goal.surface_normal_start

tf,toEE = updatetf(start_location.vector.y)
print(tf)
print(new)
print(toEE)
toEE, q = cobot._tf.T_A_B('hrr_cobot.ee_link', 'hrr_cobot.shaftgrinder_tip')
print(toEE)

   0.707107   -0.707107    0           0.455        
   0           0           1           0.04829      
  -0.707107   -0.707107    0           0.65335      
   0           0           0           1            

   0.707107    0           0.707107    0.455        
  -0.707107    0           0.707107    0.04829      
   0          -1           0           0.65335      
   0           0           0           1            

[ 0.     -0.1555 -0.0945]
[0.06682 0.06682 0.1555 ]


In [66]:
toEE = np.array([0., -0.156, 0.094])
#toEE = np.array([0.06682, -0.06682, 0.1555 ])
toEE = np.array([0, -0., 0. ])

PrePoseA = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(surface_normal_start.vector),
                                                   p_location=hrr_cm.vec32np(
                                                       start_location.vector),
                                                   C_p_CE=toEE, R_C_E=tf.A[:3, :3],
                                                   safety_distance=-0.05)
print(PrePoseA)

   0.707107   -0.707107    0           0.4555       
   0           0           1           0.0159017    
  -0.707107   -0.707107    0           0.70335      
   0           0           0           1            



In [60]:
cobot.move_to_pose(PrePoseA,v_max=0.02,rot_max=0.05)
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1646066841.033745]: reached goal pose at step 477 / 5000


In [8]:
cobot.cobot_state_msg("cutting", status=CobotState.BUSY)

stamp: 
  secs: 1641306398
  nsecs: 770958185
ctrl: 0
state: 126
tool: ''
primitive: "cutting"

In [ ]:
toolmatrix = sm.SE3(np.array([[1, 0, 0, 0],
                       [0, 0, 1, -0.002],
                       [0, 1, 0, -0.001],
                       [0,0,0,1]]))
print(toolmatrix)
C = np.array([[-0.61, -0.61, 0.5, 0.43],
             [-0.32, -0.388, -0.864, 0.133],
             [ 0.717, -0.7, 0.04, 0.77],
             [0, 0, 0, 1]])
B = cobot.FK(cobot.q)
print(B)
B = np.array([[-1, 0, 0, 0.4355],
            [0, 1, 0, 0],
            [0, 0, -1, 0.70],
            [0,0,0,1]])
print(B)
A = C @ np.linalg.inv(B)
print(A)

print(type(A@B))
X = A@B
print(type(X))

geherunter = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, -0.05],
                       [0,0,0,1]])

neu = geherunter @ X
print(neu)
#[[ 0.61,   -0.61,   -0.5,      0.51435],
# [ 0.32,    -0.388,    0.864,   -0.61116]
# [-0.717,   -0.7,     -0.04,     1.11025]
# [ 0.,       0.,       0.,       1.     ]]
#newposex = sm.SE3(X)

In [ ]:
## calibrate position without shaft grinder

calibpose = np.array([[-1, 0, 0, 0.4355],
            [0, 1, 0, 0],
            [0, 0, -1, 0.70],
            [0,0,0,1]])
print(calibpose)

geherunter = np.array([[1, 0, 0, 0.21],
                       [0, 1, 0, 0.19],
                       [0, 0, 1, -0.6],
                       [0,0,0,1]])

neucalib = geherunter @ calibpose
print(neucalib)
print(cobot.q)
print(cobot)
print(cobot.sns_pos)
print(cobot.FK(cobot.q))
#[[ 0.61,   -0.61,   -0.5,      0.51435],
# [ 0.32,    -0.388,    0.864,   -0.61116]
# [-0.717,   -0.7,     -0.04,     1.11025]
# [ 0.,       0.,       0.,       1.     ]]
#newposex = sm.SE3(X)

In [ ]:
## calibrate position without shaft grinder

calibpose = np.array([[-1, 0, 0, 0.4355],
            [0, 1, 0, 0],
            [0, 0, -1, 0.70],
            [0,0,0,1]])
print(calibpose)

geherunter = np.array([[1, 0, 0, 0.21],
                       [0, 1, 0, 0.19],
                       [0, 0, 1, -0.6],
                       [0,0,0,1]])

neucalib = geherunter @ calibpose
print(neucalib)
print(cobot.q)
print(cobot)
print(cobot.sns_pos)
print(cobot.FK(cobot.q))
#[[ 0.61,   -0.61,   -0.5,      0.51435],
# [ 0.32,    -0.388,    0.864,   -0.61116]
# [-0.717,   -0.7,     -0.04,     1.11025]
# [ 0.,       0.,       0.,       1.     ]]
#newposex = sm.SE3(X)

In [ ]:
## calib with shaft grinder

C = np.array([[-0.61, -0.61, 0.5, 0.43],
             [-0.32, -0.388, -0.864, 0.133],
             [ 0.717, -0.7, 0.04, 0.77],
             [0, 0, 0, 1]])
B = cobot.FK(cobot.q)
print(B)
B = np.array([[-1, 0, 0, 0.4355],
            [0, 1, 0, 0],
            [0, 0, -1, 0.70],
            [0,0,0,1]])
print(B)
A = C @ np.linalg.inv(B)
print(A)

print(type(A@B))
X = A@B
print(type(X))

geherunter = np.array([[1, 0, 0, 0.155],
                       [0, 1, 0, 0.14],
                       [0, 0, 1, -0.64],
                       [0,0,0,1]])

neu = geherunter @ X
print(neu)
#[[ 0.61,   -0.61,   -0.5,      0.51435],
# [ 0.32,    -0.388,    0.864,   -0.61116]
# [-0.717,   -0.7,     -0.04,     1.11025]
# [ 0.,       0.,       0.,       1.     ]]
#newposex = sm.SE3(X)

In [ ]:
X1 = np.array([[-0.614028 ,   -0.608811,     0.502313,      0.375 ],
 [-0.321531,    -0.388273,   -0.863633,    0.083],
 [ 0.720824,   -0.691804,      0.0426591,     0.73],
 [ 0.,       0.  ,     0.   ,    1.     ]])
print(X)

In [ ]:
X2 = np.array([[0.727097 ,   0.675341,     0.123469,      0.375 ],
 [0.0743252,    0.101354,   -0.99207,    0.083],
 [ -0.6825,   0.730508,      0.0234991,     0.73],
 [ 0.,       0.  ,     0.   ,    1.     ]])
print(X2)
print(cobot.FK(cobot.q))

In [ ]:
cobot.goTo(X2)
print(cobot.FK(cobot.q))

In [5]:
shaftgrinder = hrr_rob.grinder_server()

AttributeError: module 'hrr_cobot_robot' has no attribute 'grinder_server'

In [85]:
print(cobot.FK(cobot.q))
gotoPointnew = np.array([[1, 0, 0, 0.15],
                       [0, 1, 0, 0.077],
                       [0, 0, 1, -0.035],
                       [0,0,0,1]])
ausdemWeg = np.array([[-0.439919, -0.439733, 0.783011, 0.555965],
                       [0.552695, 0.554649, 0.622007, -0.151232],
                       [-0.707813, 0.706399, 0, 0.18],
                       [0,0,0,1]])
pointZ = gotoPointnew @ ausdemWeg
print(pointZ)

  -0.440004   -0.43861     0.783592    0.675489     
   0.552408    0.555758    0.621271   -0.0747128    
  -0.707984    0.706225   -0.00224414  0.145473     
   0           0           0           1            

[[-0.43992 -0.43973  0.78301  0.70597]
 [ 0.5527   0.55465  0.62201 -0.07423]
 [-0.70781  0.7064   0.       0.145  ]
 [ 0.       0.       0.       1.     ]]


In [87]:
cobot.move_to_pose(pointZ,v_max=0.001,rot_max=0.001)
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1639066854.162032]: reached goal pose at step 1209 / 5000


## Test with Planner

In [59]:
cobot.tool_controller
cobot.tool_controller.rpm = 0
cobot.change_tool(None)
cobot.change_tool("shaftgrinder")
cobot.tool_controller.reset()

[WARN] [1641377931.325951]: minimum value is 3500 => set value to 0


In [30]:
def MoveLoop(pose,v_max=0.005,rot_max=0.005,timeout=50,rpm=0.005):
    
    cobot.move_to_pose(pose,v_max,rot_max,rot_precision_threshold=0.001, pos_precision_threshold=0.001)
    T = int(timeout * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        if rpm != 0:
            cobot.tool_controller.enable_grinder()
        else:
            cobot.tool_controller.disable()
        cobot.update()

In [75]:
Shaftgrinder_tf = np.array([[-0.707107, 0.707107, 0.238541, 0.455498],
                            [0, 0, 1, 0.0482934],
                            [0.707107, 0.707107, 0, 0.653348],
                            [0, 0, 0, 1]])
toEE = np.array([0,0,0])
#toEE = np.array([0.036878, 0.1479008, -0.107603]) for coordinate calculation. for testing not needed

In [76]:
material= MaterialType()
material = material.PLASTIC

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.4555,0.0159017,0.65335], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.4555,-0.0540378,0.65335], "pointB")

In [16]:
cobot.tool_controller
cobot.change_tool("shaftgrinder")
cobot.T_B_C_robot

SE3:  -0.0143648   0.232998    0.972371    0.59606      
       0.00222875  0.972476   -0.232991    0.0697168    
      -0.999894   -0.0011797  -0.0144887   0.701112     
       0           0           0           1            
    

In [17]:
cobot.tool_controller.rpm = 4000

In [77]:
PrePoseA = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalA.vector),
                                        p_location=hrr_cm.vec32np(pointA.vector),
                                        C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                        safety_distance=-0.05)

PrePoseB = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalB.vector),
                                       p_location=hrr_cm.vec32np(pointB.vector),
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                       safety_distance=-0.05)

CuttingPoseA = hrr_cm.calc_EE_pre_pose(normal=hrr_cm.vec32np(normalA.vector),
                                       p_location=hrr_cm.vec32np(pointA.vector),
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                       safety_distance=0.005)

CuttingPoseB = hrr_cm.calc_EE_pre_pose(normal=hrr_cm.vec32np(normalB.vector),
                                       p_location=hrr_cm.vec32np(pointB.vector),
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                       safety_distance=0.005)

# for planner:
pre_location_quat = hrr_cm.homog2quat(PrePoseA)
pre_location_pos = hrr_cm.homog2pos(PrePoseA)

print(pre_location_pos)
print(pre_location_quat)

[0.49238 0.1638  0.59575]
quaternion(-0.187721339376569, 0.340781349598501, 0.619570877112396, 0.681733598073813)


In [67]:
 # Initialize the planner action client.
    
# >> Remeber to run the planner node (rosrun planner planner_server_v2.py) in Robot PC terminal. Wait for the message to appear "initialization done"
# >> 
#rospy.init_node('planner_client')

client1 = actionlib.SimpleActionClient('planner', PlannerAction)
client1.wait_for_server()

# Sending deride end configuration
goal = PlannerGoal()
goal.goal_pose.header.frame_id = "map"
goal.goal_pose.header.stamp = rospy.Time.now()

In [68]:
# pose for the planner

#position of end_configuration
goal.goal_pose.pose.position.x = pre_location_pos[0]
goal.goal_pose.pose.position.y = pre_location_pos[1]
goal.goal_pose.pose.position.z = pre_location_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = pre_location_quat.x
goal.goal_pose.pose.orientation.y = pre_location_quat.y
goal.goal_pose.pose.orientation.z = pre_location_quat.z
goal.goal_pose.pose.orientation.w = pre_location_quat.w

In [69]:
 client1.send_goal(goal)

In [78]:
# Action

## Planner here: 
#MoveLoop(pose=PrePoseA,v_max=0.05,rot_max=0.2,timeout=50,rpm=0)

## MoveLoop here
MoveLoop(pose=CuttingPoseA,v_max=0.05,rot_max=0.3,timeout=50,rpm=4000)
MoveLoop(pose=CuttingPoseB,v_max=0.05,rot_max=0.3,timeout=50,rpm=4000)
MoveLoop(pose=PrePoseB,v_max=0.05,rot_max=0.3,timeout=50,rpm=4000)
cobot.tool_controller.disable()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1641378925.824804]: reached goal pose at step 4298 / 5000


  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1641378959.735219]: reached goal pose at step 3381 / 5000


  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1641378991.746532]: reached goal pose at step 3190 / 5000


In [61]:
cobot.tool_controller.rpm = 4000
cobot.tool_controller.enable_grinder()

In [62]:
cobot.tool_controller.disable()

In [60]:
cobot.tool_controller.enable_grinder()

## Test Functionality

In [41]:
def Move_to_all_Poses(PrePoseA,CuttingPoseA,CuttingPoseB,PrePoseB,rpm):
    
    """
    Executes all MoveLoops: from PrePoseA to CuttingPoseA to CuttingPoseB to PrePoseB
    
    Args: 
    PrePoseA (SE3): Starting pose above cutting surface (offset to cutting surface in defined normal direction)
    CuttingPoseA (SE3): Start Position for Cutting (already in material with defined depth)
    CuttingPoseB (SE3): End Position for Cutting (in material with defined depth)
    PrePoseB (SE3): End Pose above cutting surface (offset to cutting surface in defined normal direction)
    
    
    """

    MoveLoop(pose=PrePoseA,v_max=0.2,rot_max=0.4,timeout=5000,rpm=0)
    MoveLoop(pose=CuttingPoseA,v_max=0.06,rot_max=0.02,timeout=5000,rpm=rpm)
    MoveLoop(pose=CuttingPoseB,v_max=0.06,rot_max=0.02,timeout=5000,rpm=rpm)
    MoveLoop(pose=PrePoseB,v_max=0.2,rot_max=0.4,timeout=5000,rpm=rpm)

    cobot.tool_controller.rpm = 0
    #cobot.tool_controller.disable()

In [42]:
def points_to_poses(normalA,pointA,normalB,pointB,Shaftgrinder_tf,cuttingdepth=0.005,safety_distance=0.05,toEE=[0,0,0]):

    PrePoseA = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalA.vector),
                                            p_location=hrr_cm.vec32np(pointA.vector),
                                            C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                            safety_distance=-safety_distance)

    PrePoseB = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalB.vector),
                                           p_location=hrr_cm.vec32np(pointB.vector),
                                           C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                           safety_distance=-safety_distance)

    CuttingPoseA = hrr_cm.calc_EE_pre_pose(normal=hrr_cm.vec32np(normalA.vector),
                                            p_location=hrr_cm.vec32np(pointA.vector),
                                            C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                            safety_distance=cuttingdepth)

    CuttingPoseB = hrr_cm.calc_EE_pre_pose(normal=hrr_cm.vec32np(normalB.vector),
                                           p_location=hrr_cm.vec32np(pointB.vector),
                                           C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                           safety_distance=cuttingdepth)
    
    return PrePoseA, PrePoseB, CuttingPoseA, CuttingPoseB


In [43]:
def MoveLoop(pose,v_max=0.005,rot_max=0.005,timeout=50,rpm=0.005):
    
    cobot.move_to_pose(pose,v_max,rot_max,rot_precision_threshold=0.001, pos_precision_threshold=0.001)
    T = int(timeout * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        if rpm != 0:
            cobot.tool_controller.rpm = rpm
            cobot.tool_controller.enable_grinder()
        else:
            cobot.tool_controller.rpm = 0
            #cobot.tool_controller.disable()
        cobot.update()

In [44]:
print(cobot.FK(cobot.q))

  -0.706413    0.707799    0.00096377  0.591832     
  -9.77023e-06-0.00137139  0.999999   -0.199711     
   0.7078      0.706412    0.000975684 0.415999     
   0           0           0           1            



In [45]:
Shaftgrinder_tf = (cobot.FK(cobot.q).A)
toEE = np.array([0,0,0])
#toEE = np.array([0.036878, 0.1479008, -0.107603]) for coordinate calculation. for testing not needed
print(Shaftgrinder_tf)

[[-0.70641  0.7078   0.00096  0.59183]
 [-0.00001 -0.00137  1.      -0.19971]
 [ 0.7078   0.70641  0.00098  0.416  ]
 [ 0.       0.       0.       1.     ]]


In [17]:
cobot.tool_controller

In [60]:
cobot.change_tool("shaftgrinder")

True

In [32]:
cobot.change_tool(None)

True

In [46]:
cobot.tool_controller.enable_grinder()

In [58]:
cobot.tool_controller.disable()

In [62]:
cobot.tool_controller.rpm = 0

[WARN] [1644845218.134773]: minimum value is 3500 => set value to 0


### MW Oven

In [48]:
cobot.tool_controller.rpm = 0

[WARN] [1644844835.305933]: minimum value is 3500 => set value to 0


In [49]:
# Shaftgrinder_tf_MW = np.array([[-0.66991,  0.64932,  0.35999,  0.48663],
#  [ 0.25544, -0.2537,   0.93295, -0.12696],
#  [ 0.69711,  0.71695,  0.0041,   0.53644],
#  [ 0. ,      0. ,      0.  ,     1.     ]])
Shaftgrinder_tf_MW = Shaftgrinder_tf
print(Shaftgrinder_tf_MW)
toEE = np.array([0,0,0])

[[-0.70641  0.7078   0.00096  0.59183]
 [-0.00001 -0.00137  1.      -0.19971]
 [ 0.7078   0.70641  0.00098  0.416  ]
 [ 0.       0.       0.       1.     ]]


In [49]:
cobot.FK(cobot.q)

SE3:  -0.706415    0.707798    0.000966046 0.45599      
      -8.48723e-06-0.00137333  0.999999    0.0478077    
       0.707798    0.706414    0.000976148 0.653832     
       0           0           0           1            
    

In [50]:
material= MaterialType()
material = material.PLASTIC

xstartMW = 0.592829 #determine start value next to cable (in start cutting pose)
ystartMW = -0.229713
zstartMW =  0.367

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartMW,ystartMW,zstartMW], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartMW,ystartMW+0.031,zstartMW], "pointB")

In [51]:
PrePoseA = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalA.vector),
                                        p_location=hrr_cm.vec32np(pointA.vector),
                                        C_p_CE=toEE, R_C_E=Shaftgrinder_tf_MW[:3, :3],
                                        safety_distance=-0.05)

PrePoseB = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalB.vector),
                                       p_location=hrr_cm.vec32np(pointB.vector),
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf_MW[:3, :3],
                                       safety_distance=-0.05)

CuttingPoseA = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalA.vector),
                                        p_location=hrr_cm.vec32np(pointA.vector),
                                        C_p_CE=toEE, R_C_E=Shaftgrinder_tf_MW[:3, :3],
                                        safety_distance=-0)

CuttingPoseB = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalB.vector),
                                       p_location=hrr_cm.vec32np(pointB.vector),
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf_MW[:3, :3],
                                       safety_distance=-0)

In [56]:
print(CuttingPoseA)
print(cobot.FK(cobot.q))
print(cobot.F_max)
cobot.F_max = 110
print(cobot.F_max)

  -0.706413    0.707799    0.000963933 0.592829     
  -9.69813e-06-0.00137155  0.999999   -0.229713     
   0.7078      0.706412    0.000975745 0.367        
   0           0           0           1            

  -0.706417    0.707795    0.000966643 0.591833     
  -2.128e-06  -0.00136783  0.999999   -0.228718     
   0.707796    0.706417    0.000967767 0.41373      
   0           0           0           1            

80.0
110


In [63]:
MoveLoop(pose=PrePoseA,v_max=0.2,rot_max=0.4,timeout=5000,rpm=0)
MoveLoop(pose=CuttingPoseA,v_max=0.06,rot_max=0.02,timeout=5000,rpm=5000)
MoveLoop(pose=CuttingPoseB,v_max=0.06,rot_max=0.02,timeout=5000,rpm=5000)
MoveLoop(pose=PrePoseB,v_max=0.2,rot_max=0.4,timeout=5000,rpm=5000)
cobot.tool_controller.rpm = 0

  0%|          | 0/500000 [00:00<?, ?it/s]

[WARN] [1644845235.485852]: minimum value is 3500 => set value to 0
[WARN] [1644845235.497886]: minimum value is 3500 => set value to 0
[WARN] [1644845235.507999]: minimum value is 3500 => set value to 0
[WARN] [1644845235.520360]: minimum value is 3500 => set value to 0
[WARN] [1644845235.528986]: minimum value is 3500 => set value to 0
[WARN] [1644845235.538007]: minimum value is 3500 => set value to 0
[WARN] [1644845235.548063]: minimum value is 3500 => set value to 0
[WARN] [1644845235.557983]: minimum value is 3500 => set value to 0
[WARN] [1644845235.567940]: minimum value is 3500 => set value to 0
[WARN] [1644845235.578229]: minimum value is 3500 => set value to 0
[WARN] [1644845235.588030]: minimum value is 3500 => set value to 0
[WARN] [1644845235.598029]: minimum value is 3500 => set value to 0
[WARN] [1644845235.608235]: minimum value is 3500 => set value to 0
[WARN] [1644845235.622716]: minimum value is 3500 => set value to 0
[WARN] [1644845235.634115]: minimum value is 350

[WARN] [1644845236.754961]: minimum value is 3500 => set value to 0
[WARN] [1644845236.764791]: minimum value is 3500 => set value to 0
[WARN] [1644845236.774081]: minimum value is 3500 => set value to 0
[WARN] [1644845236.783870]: minimum value is 3500 => set value to 0
[WARN] [1644845236.793154]: minimum value is 3500 => set value to 0
[WARN] [1644845236.799391]: minimum value is 3500 => set value to 0
[WARN] [1644845236.808024]: minimum value is 3500 => set value to 0
[WARN] [1644845236.817940]: minimum value is 3500 => set value to 0
[WARN] [1644845236.827909]: minimum value is 3500 => set value to 0
[WARN] [1644845236.837955]: minimum value is 3500 => set value to 0
[WARN] [1644845236.847941]: minimum value is 3500 => set value to 0
[WARN] [1644845236.857901]: minimum value is 3500 => set value to 0
[WARN] [1644845236.867938]: minimum value is 3500 => set value to 0
[WARN] [1644845236.878048]: minimum value is 3500 => set value to 0
[WARN] [1644845236.889780]: minimum value is 350

[WARN] [1644845238.000777]: minimum value is 3500 => set value to 0
[WARN] [1644845238.012331]: minimum value is 3500 => set value to 0
[WARN] [1644845238.020009]: minimum value is 3500 => set value to 0
[WARN] [1644845238.028375]: minimum value is 3500 => set value to 0
[WARN] [1644845238.038029]: minimum value is 3500 => set value to 0
[WARN] [1644845238.047936]: minimum value is 3500 => set value to 0
[WARN] [1644845238.058045]: minimum value is 3500 => set value to 0
[WARN] [1644845238.067918]: minimum value is 3500 => set value to 0
[WARN] [1644845238.078028]: minimum value is 3500 => set value to 0
[WARN] [1644845238.088083]: minimum value is 3500 => set value to 0
[WARN] [1644845238.099669]: minimum value is 3500 => set value to 0
[WARN] [1644845238.114939]: minimum value is 3500 => set value to 0
[WARN] [1644845238.123454]: minimum value is 3500 => set value to 0
[WARN] [1644845238.129246]: minimum value is 3500 => set value to 0
[WARN] [1644845238.137888]: minimum value is 350

[WARN] [1644845239.258503]: minimum value is 3500 => set value to 0
[WARN] [1644845239.267983]: minimum value is 3500 => set value to 0
[WARN] [1644845239.278038]: minimum value is 3500 => set value to 0
[WARN] [1644845239.288101]: minimum value is 3500 => set value to 0
[WARN] [1644845239.297967]: minimum value is 3500 => set value to 0
[WARN] [1644845239.310103]: minimum value is 3500 => set value to 0
[WARN] [1644845239.318864]: minimum value is 3500 => set value to 0
[WARN] [1644845239.328228]: minimum value is 3500 => set value to 0
[WARN] [1644845239.338032]: minimum value is 3500 => set value to 0
[WARN] [1644845239.347900]: minimum value is 3500 => set value to 0
[WARN] [1644845239.357971]: minimum value is 3500 => set value to 0
[WARN] [1644845239.367971]: minimum value is 3500 => set value to 0
[WARN] [1644845239.378082]: minimum value is 3500 => set value to 0
[WARN] [1644845239.388074]: minimum value is 3500 => set value to 0
[WARN] [1644845239.397971]: minimum value is 350

...

  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1644845346.308421]: reached goal pose at step 4528 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1644845362.041609]: reached goal pose at step 1554 / 500000


[WARN] [1644845362.051117]: minimum value is 3500 => set value to 0


In [59]:
cobot.tool_controller.rpm = 0

[WARN] [1644845113.825723]: minimum value is 3500 => set value to 0


In [54]:
cobot.move_to_pose(Shaftgrinder_tf_1,v_max=0.01,rot_max=0.01)

NameError: name 'Shaftgrinder_tf_1' is not defined

In [294]:
cobot.move_to_pose(PrePoseB,v_max=0.01,rot_max=0.01)

In [316]:
# executes the loop to move the robot
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1642071689.516240]: reached goal pose at step 1571 / 5000


[WARN] [1642072215.984281]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1642072215.994999]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1642072215.997932]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


### Emergency light

#### Clips

In [17]:
# Shaftgrinder_tf_ELC = np.array([[-0.66991,  0.64932,  0.35999,  0.48663],
#  [ 0.25544, -0.2537,   0.93295, -0.12696],
#  [ 0.69711,  0.71695,  0.0041,   0.53644],
#  [ 0. ,      0. ,      0.  ,     1.     ]])
Shaftgrinder_tf_ELC = Shaftgrinder_tf
print(Shaftgrinder_tf_ELC)
toEE = np.array([0,0,0])

[[-0.70759  0.70663 -0.0006   0.62456]
 [-0.00004  0.00081  1.      -0.00203]
 [ 0.70663  0.70759 -0.00055  0.26897]
 [ 0.       0.       0.       1.     ]]


In [18]:
print(cobot.FK(cobot.q))

  -0.707585    0.706628   -0.000599266 0.624559     
  -3.66942e-05 0.00081132  1          -0.00203192   
   0.706628    0.707585   -0.000548149 0.268974     
   0           0           0           1            



In [22]:
cobot.tool_controller.rpm = 20000

In [61]:
cobot.tool_controller.enable_grinder()

In [62]:
cobot.tool_controller.disable()

In [74]:
cobot.FK(cobot.q)

SE3:  -0.70795     0.706262   -0.000602564 0.463575     
       0.000131847 0.000985336 1          -0.301272     
       0.706262    0.70795    -0.000790687 0.201447     
       0           0           0           1            
    

In [57]:
material= MaterialType()
material = material.PLASTIC

xstartELC = 0.4645
ystartELC = -0.301272 
zstartELC = 0.201447 


normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")

pointA1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC,ystartELC,zstartELC], "pointA1")
pointB1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC,ystartELC+0.012,zstartELC], "pointB1")

pointA2 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC,ystartELC+0.149,zstartELC], "pointA2")
pointB2 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC,ystartELC+0.012+0.149,zstartELC], "pointB2")

pointA3 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC,ystartELC+0.298,zstartELC], "pointA3")
pointB3 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC,ystartELC+0.012+0.298,zstartELC], "pointB3")

pointA4 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC+0.1635,ystartELC+0.298,zstartELC], "pointA4")
pointB4 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC+0.1635,ystartELC+0.012+0.298,zstartELC], "pointB4")

pointA5 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC+0.1635,ystartELC+0.149,zstartELC], "pointA5")
pointB5 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC+0.1635,ystartELC+0.012+0.149,zstartELC], "pointB5")

pointA6 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC+0.1635,ystartELC,zstartELC], "pointA6")
pointB6 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELC+0.1635,ystartELC+0.012,zstartELC], "pointB6")


In [58]:
PrePoseA1, PrePoseB1, CuttingPoseA1, CuttingPoseB1 = points_to_poses(normalA=normalA, normalB=normalB, pointA=pointA1, pointB=pointB1, 
                                                                     safety_distance=0.05, cuttingdepth=0.017, 
                                                                     Shaftgrinder_tf = Shaftgrinder_tf_ELC, toEE=toEE)

PrePoseA2, PrePoseB2, CuttingPoseA2, CuttingPoseB2 = points_to_poses(normalA=normalA, normalB=normalB, pointA=pointA2, pointB=pointB2,
                                                                    safety_distance=0.05, cuttingdepth=0.017,
                                                                    Shaftgrinder_tf = Shaftgrinder_tf_ELC, toEE=toEE)

PrePoseA3, PrePoseB3, CuttingPoseA3, CuttingPoseB3 = points_to_poses(normalA=normalA, normalB=normalB, pointA=pointA3, pointB=pointB3,
                                                                     safety_distance=0.05, cuttingdepth=0.017,
                                                                    Shaftgrinder_tf = Shaftgrinder_tf_ELC, toEE=toEE)


PrePoseA4, PrePoseB4, CuttingPoseA4, CuttingPoseB4 = points_to_poses(normalA=normalA, normalB=normalB, pointA=pointA4, pointB=pointB4,
                                                                    safety_distance=0.05, cuttingdepth=0.017,
                                                                    Shaftgrinder_tf = Shaftgrinder_tf_ELC,toEE=toEE)

PrePoseA5, PrePoseB5, CuttingPoseA5, CuttingPoseB5 = points_to_poses(normalA=normalA, normalB=normalB, pointA=pointA5, pointB=pointB5,
                                                                    safety_distance=0.05, cuttingdepth=0.017,
                                                                    Shaftgrinder_tf = Shaftgrinder_tf_ELC, toEE=toEE)

PrePoseA6, PrePoseB6, CuttingPoseA6, CuttingPoseB6 = points_to_poses(normalA=normalA, normalB=normalB, pointA=pointA6, pointB=pointB6,
                                                                    safety_distance=0.05, cuttingdepth=0.017,
                                                                    Shaftgrinder_tf = Shaftgrinder_tf_ELC, toEE=toEE)

In [24]:
cobot.move_to_pose(PrePoseA1,v_max=0.01,rot_max=0.01)

In [31]:
cobot.move_to_pose(Shaftgrinder_tf_1,v_max=0.01,rot_max=0.01)

In [44]:
cobot.F_max = 120

In [22]:
print(CuttingPoseA1)
print(PrePoseB1)

  -0.708026    0.706186   -0.000495976 0.473603     
   0.000275104 0.000978151 0.999999   -0.302272     
   0.706187    0.708025   -0.000886831 0.184092     
   0           0           0           1            

  -0.708026    0.706186   -0.000495976 0.473603     
   0.000275104 0.000978151 0.999999   -0.290272     
   0.706187    0.708025   -0.000886831 0.249092     
   0           0           0           1            



In [23]:
print(cobot.FK(cobot.q))

  -0.708026    0.706186   -0.000495976 0.473603     
   0.000275104 0.000978151 0.999999   -0.302272     
   0.706187    0.708025   -0.000886831 0.199092     
   0           0           0           1            



In [63]:
Move_to_all_Poses(PrePoseA=PrePoseA1,PrePoseB=PrePoseB1,CuttingPoseA=CuttingPoseA1,CuttingPoseB=CuttingPoseB1,rpm=17000)

Move_to_all_Poses(PrePoseA=PrePoseA2,PrePoseB=PrePoseB2,CuttingPoseA=CuttingPoseA2,CuttingPoseB=CuttingPoseB2,rpm=17000)

Move_to_all_Poses(PrePoseA=PrePoseA3,PrePoseB=PrePoseB3,CuttingPoseA=CuttingPoseA3,CuttingPoseB=CuttingPoseB3,rpm=17000)

Move_to_all_Poses(PrePoseA=PrePoseA4,PrePoseB=PrePoseB4,CuttingPoseA=CuttingPoseA4,CuttingPoseB=CuttingPoseB4,rpm=17000)

Move_to_all_Poses(PrePoseA=PrePoseA5,PrePoseB=PrePoseB5,CuttingPoseA=CuttingPoseA5,CuttingPoseB=CuttingPoseB5,rpm=17000)

Move_to_all_Poses(PrePoseA=PrePoseA6,PrePoseB=PrePoseB6,CuttingPoseA=CuttingPoseA6,CuttingPoseB=CuttingPoseB6,rpm=17000)

  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642001429.084635]: reached goal pose at step 1220 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642001457.240581]: reached goal pose at step 2809 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

...

[INFO] [1642001771.818265]: reached goal pose at step 2790 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642001787.878949]: reached goal pose at step 1600 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642001796.278923]: reached goal pose at step 834 / 500000


In [69]:
cobot.tool_controller.disable()

In [29]:
cobot.move_to_pose(PrePoseA6,v_max=0.01,rot_max=0.01)

In [50]:
cobot.state

'cart_servo'

In [64]:
cobot.move_to_pose(Shaftgrinder_tf_1,v_max=0.01,rot_max=0.01)

In [65]:
# executes the loop to move the robot
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1642001824.071844]: reached goal pose at step 2192 / 5000


#### Cable

In [17]:
# Shaftgrinder_tf_ELK = np.array([[-0.66991,  0.64932,  0.35999,  0.48663],
#  [ 0.25544, -0.2537,   0.93295, -0.12696],
#  [ 0.69711,  0.71695,  0.0041,   0.53644],
#  [ 0. ,      0. ,      0.  ,     1.     ]])
Shaftgrinder_tf_ELK = Shaftgrinder_tf

print(Shaftgrinder_tf_ELK)
toEE = np.array([0,0,0])

[[-0.70739  0.70682 -0.00088  0.50726]
 [-0.00038  0.00086  1.      -0.17459]
 [ 0.70682  0.70739 -0.00034  0.17061]
 [ 0.       0.       0.       1.     ]]


In [67]:
cobot.tool_controller.rpm = 20000

In [70]:
cobot.tool_controller.disable()

In [69]:
cobot.tool_controller.enable_grinder()

In [30]:
cobot.FK(cobot.q)

SE3:  -0.707489    0.706723   -0.000849726 0.508603     
      -0.000136309 0.00106589  0.999999   -0.174949     
       0.706724    0.707489   -0.000657772 0.172062     
       0           0           0           1            
    

In [64]:
material= MaterialType()
material = material.PLASTIC

#coordinates: next to cable on above cable level richtige Höhe??, check if in x oder y Richtung, depth = 8mm!

xstartELK = 0.508603 
ystartELK = -0.176949 
zstartELK = 0.172062


normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")

pointA1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELK,ystartELK,zstartELK], "pointA1")
pointB1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELK+0.014,ystartELK,zstartELK], "pointB1")

In [65]:
PrePoseA1, PrePoseB1, CuttingPoseA1, CuttingPoseB1 = points_to_poses(normalA=normalA, normalB=normalB, pointA=pointA1, pointB=pointB1, 
                                                                     cuttingdepth=0, safety_distance=0.05,
                                                                    Shaftgrinder_tf = Shaftgrinder_tf_ELK,toEE=toEE)

In [73]:
Move_to_all_Poses(PrePoseA=PrePoseA1,PrePoseB=PrePoseB1,CuttingPoseA=CuttingPoseA1,CuttingPoseB=CuttingPoseB1,rpm=20000)

  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642004357.899859]: reached goal pose at step 1243 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642004384.105376]: reached goal pose at step 2618 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642004402.193748]: reached goal pose at step 1806 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642004409.998335]: reached goal pose at step 775 / 500000


In [29]:
print(CuttingPoseB1)
print(cobot.FK(cobot.q))

  -0.707393    0.70682    -0.000876985 0.457482     
  -0.000382894 0.000857543 1          -0.186461     
   0.70682     0.707393   -0.000335983 0.564378     
   0           0           0           1            

  -0.707394    0.706819   -0.000875706 0.443961     
  -0.000379087 0.000859544 1          -0.18599      
   0.706819    0.707394   -0.00034009  0.614858     
   0           0           0           1            



In [74]:
cobot.move_to_pose(Shaftgrinder_tf_1,v_max=0.01,rot_max=0.01)

In [42]:
cobot.move_to_pose(PrePoseB1,v_max=0.01,rot_max=0.01)

In [75]:
# executes the loop to move the robot
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1642004436.580241]: reached goal pose at step 2309 / 5000


### PC

#### Clip

In [250]:
# Shaftgrinder_tf_ELK = np.array([[-0.66991,  0.64932,  0.35999,  0.48663],
#  [ 0.25544, -0.2537,   0.93295, -0.12696],
#  [ 0.69711,  0.71695,  0.0041,   0.53644],
#  [ 0. ,      0. ,      0.  ,     1.     ]])
Shaftgrinder_tf_ELK = Shaftgrinder_tf

print(Shaftgrinder_tf_ELK)
toEE = np.array([0,0,0])

[[-0.70751  0.70671 -0.00092  0.5528 ]
 [-0.00018  0.00112  1.       0.01391]
 [ 0.70671  0.70751 -0.00066  0.30837]
 [ 0.       0.       0.       1.     ]]


In [18]:
cobot.tool_controller.rpm = 25000

In [277]:
cobot.tool_controller.disable()

In [272]:
cobot.tool_controller.enable_grinder()

In [20]:
cobot.FK(cobot.q)

SE3:  -0.707508    0.706705   -0.000915601 0.5528       
      -0.000179161 0.00111622  0.999999    0.0139121    
       0.706706    0.707507   -0.000663123 0.308369     
       0           0           0           1            
    

In [257]:
material= MaterialType()
material = material.PLASTIC

#coordinates: next to cable on above cable level richtige Höhe??, check if in x oder y Richtung, depth = 8mm!

xstartELK = 0.5528 
ystartELK = 0.011
zstartELK = 0.303


normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")

pointA1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELK,ystartELK,zstartELK], "pointA1")
pointB1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartELK+0.06,ystartELK,zstartELK], "pointB1")

In [258]:
PrePoseA1, PrePoseB1, CuttingPoseA1, CuttingPoseB1 = points_to_poses(normalA=normalA, normalB=normalB, pointA=pointA1, pointB=pointB1, 
                                                                     cuttingdepth=0, safety_distance=0.05,
                                                                    Shaftgrinder_tf = Shaftgrinder_tf_ELK,toEE=toEE)

In [276]:
Move_to_all_Poses(PrePoseA=PrePoseA1,PrePoseB=PrePoseB1,CuttingPoseA=CuttingPoseA1,CuttingPoseB=CuttingPoseB1,rpm=25000)

  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642069261.044560]: reached goal pose at step 1139 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642069287.270989]: reached goal pose at step 2619 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[ERROR] [1642069289.789502]: Force threshold exceeded: |F|=121.94 > 120.00. Stop motion


KeyboardInterrupt: 

In [ ]:
print(CuttingPoseB1)
print(cobot.FK(cobot.q))

In [53]:
cobot.F_max = 120

In [274]:
cobot.move_to_pose(Shaftgrinder_tf_1,v_max=0.01,rot_max=0.01)

In [265]:
cobot.move_to_pose(PrePoseB1,v_max=0.01,rot_max=0.01)

In [275]:
# executes the loop to move the robot
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1642069195.551260]: reached goal pose at step 1787 / 5000


#### Cables

First Cable Bunch

In [162]:
Shaftgrinder_tf_PC1 = (cobot.FK(cobot.q).A)
print(Shaftgrinder_tf_PC1)

[[-0.26864  0.55951  0.78408  0.50352]
 [ 0.67257 -0.47376  0.5685  -0.04761]
 [ 0.68955  0.68007 -0.24904  0.26459]
 [ 0.       0.       0.       1.     ]]


In [164]:
Shaftgrinder_tf_PC2 = np.array([[-0.26864,  0.55951,  0.78408,  0.50352],
 [ 0.67257, -0.47376,  0.5685,  -0.04761],
 [ 0.68955,  0.68007, -0.24904 , 0.26459],
 [ 0.  ,     0.  ,     0.   ,    1.     ]]
)
print(Shaftgrinder_tf_PC2)
toEE = np.array([0,0,0])

[[-0.26864  0.55951  0.78408  0.50352]
 [ 0.67257 -0.47376  0.5685  -0.04761]
 [ 0.68955  0.68007 -0.24904  0.26459]
 [ 0.       0.       0.       1.     ]]


In [170]:
PrePoseA1 = np.array([[-0.26864,  0.55951,  0.78408,  0.50352],
 [ 0.67257, -0.47376,  0.5685,  -0.04761],
 [ 0.68955,  0.68007, -0.24904 , 0.26459+0.05],
 [ 0.  ,     0.  ,     0.   ,    1.     ]]
)

PrePoseB1 = np.array([[-0.26864,  0.55951,  0.78408,  0.50352],
 [ 0.67257, -0.47376,  0.5685,  -0.04761+0.03],
 [ 0.68955,  0.68007, -0.24904 , 0.26459+0.05],
 [ 0.  ,     0.  ,     0.   ,    1.     ]]
)

CuttingPoseA1 = np.array([[-0.26864,  0.55951,  0.78408,  0.50352],
 [ 0.67257, -0.47376,  0.5685,  -0.04761],
 [ 0.68955,  0.68007, -0.24904 , 0.26459],
 [ 0.  ,     0.  ,     0.   ,    1.     ]]
)

CuttingPoseB1 = np.array([[-0.26864,  0.55951,  0.78408,  0.50352],
 [ 0.67257, -0.47376,  0.5685,  -0.04761+0.03],
 [ 0.68955,  0.68007, -0.24904 , 0.26459],
 [ 0.  ,     0.  ,     0.   ,    1.     ]]
)

In [ ]:
material= MaterialType()
material = material.PLASTIC

#coordinates: next to cable on cable level richtige Höhe, check if in x oder y Richtung

xstartPC2 = 0
ystartPC2 = 0
zstartPC2 = 0.012

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")

pointA1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartPC2,ystartPC2,zstartPC2], "pointA1")
pointB1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartPC2,ystartPC2+0.015,zstartPC2], "pointB1")

In [ ]:
PrePoseA1, PrePoseB1, CuttingPoseA1, CuttingPoseB1 = points_to_poses(normalA, normalB, pointA1, pointB1, 
                                                                     cuttingdepth=0, safety_distance=0.05,
                                                                    Shaftgrinder_tf = Shaftgrinder_tf_PC2)

In [ ]:
Move_to_all_Poses(PrePoseA=PrePoseA1,PrePoseB=PrePoseB1,CuttingPoseA=CuttingPoseA1,CuttingPoseB=CuttingPoseB1,rpm=20000)

Second cable bunch

In [138]:
Shaftgrinder_tf_PC2 = (cobot.FK(cobot.q).A)
print(Shaftgrinder_tf_PC2)
toEE = np.array([0,0,0])

[[-0.25151  0.58966  0.76749  0.50753]
 [ 0.22861 -0.73435  0.63911  0.01337]
 [ 0.94047  0.3362   0.04989  0.22234]
 [ 0.       0.       0.       1.     ]]


In [145]:
print(CuttingPoseB1)

  -0.749466   -0.576492   -0.325512    0.50753      
   0.273497   -0.717365    0.640771    0.01337      
  -0.60291     0.391209    0.695309    0.22234      
   0           0           0           1            



In [139]:
Shaftgrinder_tf_PC2 = np.array([[-0.25151,  0.58966,  0.76749,  0.50753],
 [ 0.22861, -0.73435,  0.63911,  0.01337],
 [ 0.94047,  0.3362,   0.04989,  0.22234],
 [ 0. ,      0. ,      0.  ,     1.     ]]
)

In [ ]:
cobot.tool_controller.rpm = 20000

In [ ]:
cobot.tool_controller.disable()

In [ ]:
cobot.tool_controller.enable_grinder()

In [140]:
material= MaterialType()
material = material.PLASTIC

#coordinates: next to cable on cable level richtige Höhe, check if in x oder y Richtung

xstartPC2 = 0.50753
ystartPC2 = 0.01337
zstartPC2 = 0.22234

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")

pointA1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartPC2,ystartPC2-0.04,zstartPC2], "pointA1")
pointB1 = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstartPC2,ystartPC2,zstartPC2], "pointB1")

In [153]:
PrePoseA1 = np.array([[-0.25151,  0.58966,  0.76749,  0.50753],
 [ 0.22861, -0.73435,  0.63911,  0.01337-0.05],
 [ 0.94047,  0.3362,   0.04989,  0.22234+0.05],
 [ 0. ,      0. ,      0.  ,     1.     ]]
)
PrePoseB1 = np.array([[-0.25151,  0.58966,  0.76749,  0.50753],
 [ 0.22861, -0.73435,  0.63911,  0.01337],
 [ 0.94047,  0.3362,   0.04989,  0.22234+0.05],
 [ 0. ,      0. ,      0.  ,     1.     ]]
)

CuttingPoseA1 = np.array([[-0.25151,  0.58966,  0.76749,  0.50753],
 [ 0.22861, -0.73435,  0.63911,  0.01337-0.05],
 [ 0.94047,  0.3362,   0.04989,  0.22234],
 [ 0. ,      0. ,      0.  ,     1.     ]]
)

CuttingPoseB1 = np.array([[-0.25151,  0.58966,  0.76749,  0.50753],
 [ 0.22861, -0.73435,  0.63911,  0.01337],
 [ 0.94047,  0.3362,   0.04989,  0.22234],
 [ 0. ,      0. ,      0.  ,     1.     ]]
)

In [141]:
PrePoseA1, PrePoseB1, CuttingPoseA1, CuttingPoseB1 = points_to_poses(normalA, normalB, pointA1, pointB1, 
                                                                     cuttingdepth=0, safety_distance=0.05,
                                                                    Shaftgrinder_tf = Shaftgrinder_tf_PC2)

In [ ]:
Move_to_all_Poses(PrePoseA=PrePoseA1,PrePoseB=PrePoseB1,CuttingPoseA=CuttingPoseA1,CuttingPoseB=CuttingPoseB1,rpm=20000)

In [ ]:
cobot.F_max = 120

In [ ]:
cobot.move_to_pose(Shaftgrinder_tf_1,v_max=0.01,rot_max=0.01)

In [175]:
cobot.move_to_pose(PrePoseB1,v_max=0.01,rot_max=0.01)

In [176]:
# executes the loop to move the robot
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1642066613.187943]: reached goal pose at step 776 / 5000


Record:

In [244]:
PrePoseA1 = np.array([[-0.26864,  0.55951,  0.78408,  0.50352-0.002],
 [ 0.67257, -0.47376,  0.5685,  -0.04761],
 [ 0.68955,  0.68007, -0.24904 , 0.26459+0.05],
 [ 0.  ,     0.  ,     0.   ,    1.     ]]
)

PrePoseB1 = np.array([[-0.26864,  0.55951,  0.78408,  0.50352-0.002],
 [ 0.67257, -0.47376,  0.5685,  -0.04761+0.03],
 [ 0.68955,  0.68007, -0.24904 , 0.26459+0.05],
 [ 0.  ,     0.  ,     0.   ,    1.     ]]
)

CuttingPoseA1 = np.array([[-0.26864,  0.55951,  0.78408,  0.50352-0.002],
 [ 0.67257, -0.47376,  0.5685,  -0.04761],
 [ 0.68955,  0.68007, -0.24904 , 0.26459-0.009],
 [ 0.  ,     0.  ,     0.   ,    1.     ]]
)

CuttingPoseB1 = np.array([[-0.26864,  0.55951,  0.78408,  0.50352-0.002],
 [ 0.67257, -0.47376,  0.5685,  -0.04761+0.03],
 [ 0.68955,  0.68007, -0.24904 , 0.26459-0.009],
 [ 0.  ,     0.  ,     0.   ,    1.     ]]
)

In [245]:
Move_to_all_Poses(PrePoseA=PrePoseA1,PrePoseB=PrePoseB1,CuttingPoseA=CuttingPoseA1,CuttingPoseB=CuttingPoseB1,rpm=20000)

  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642068493.323317]: reached goal pose at step 4248 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642068520.647062]: reached goal pose at step 2727 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642068543.150431]: reached goal pose at step 2244 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642068551.312343]: reached goal pose at step 809 / 500000


In [246]:
PrePoseA1 = np.array([[-0.25151,  0.58966,  0.76749,  0.50753],
 [ 0.22861, -0.73435,  0.63911,  0.01337-0.05],
 [ 0.94047,  0.3362,   0.04989,  0.22234+0.05],
 [ 0. ,      0. ,      0.  ,     1.     ]]
)
PrePoseB1 = np.array([[-0.25151,  0.58966,  0.76749,  0.50753],
 [ 0.22861, -0.73435,  0.63911,  0.01337],
 [ 0.94047,  0.3362,   0.04989,  0.22234+0.10],
 [ 0. ,      0. ,      0.  ,     1.     ]]
)

CuttingPoseA1 = np.array([[-0.25151,  0.58966,  0.76749,  0.50753],
 [ 0.22861, -0.73435,  0.63911,  0.01337-0.05],
 [ 0.94047,  0.3362,   0.04989,  0.215014 ],
 [ 0. ,      0. ,      0.  ,     1.     ]]
)

CuttingPoseB1 = np.array([[-0.25151,  0.58966,  0.76749,  0.50753],
 [ 0.22861, -0.73435,  0.63911,  0.01337],
 [ 0.94047,  0.3362,   0.04989,  0.215014 ],
 [ 0. ,      0. ,      0.  ,     1.     ]]
)

In [247]:
Move_to_all_Poses(PrePoseA=PrePoseA1,PrePoseB=PrePoseB1,CuttingPoseA=CuttingPoseA1,CuttingPoseB=CuttingPoseB1,rpm=20000)

  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642068583.175502]: reached goal pose at step 2674 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642068610.314466]: reached goal pose at step 2707 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642068636.322241]: reached goal pose at step 2593 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1642068645.672513]: reached goal pose at step 928 / 500000


In [248]:
cobot.move_to_pose(Shaftgrinder_tf_1,v_max=0.01,rot_max=0.01)

In [238]:
cobot.move_to_pose(PrePoseB1,v_max=0.01,rot_max=0.01)

In [249]:
# executes the loop to move the robot
T = int(50 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerr(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/5000 [00:00<?, ?it/s]

[INFO] [1642068688.061724]: reached goal pose at step 3957 / 5000


In [243]:
cobot.tool_controller.disable()

In [242]:
cobot.tool_controller.enable_grinder()

In [216]:
print(PrePoseB1)
print(cobot.FK(cobot.q))

[[-0.25151  0.58966  0.76749  0.50753]
 [ 0.22861 -0.73435  0.63911  0.01337]
 [ 0.94047  0.3362   0.04989  0.22234]
 [ 0.       0.       0.       1.     ]]
  -0.250798    0.590723    0.766907    0.506536     
   0.227425   -0.734099    0.639825    0.014362     
   0.940945    0.334881    0.0497654   0.215014     
   0           0           0           1            



In [1]:
import time 

In [15]:
start = time.time()
print(start)
b = 0

1644510160.1788118


In [16]:
while time.time() - start < 5 and b == 0:
    print("%alias")
    time.sleep(7)
    b = 1
    print("b")

%alias
b
